## Setup

In [1]:
# Install microdf
!pip install git+https://github.com/PSLmodels/microdf.git
!pip install git+https://github.com/MaxGhenis/scf.git

  Cloning https://github.com/PSLmodels/microdf.git to /tmp/pip-req-build-eqo2po57
  Running command git clone -q https://github.com/PSLmodels/microdf.git /tmp/pip-req-build-eqo2po57
  Created wheel for microdf: filename=microdf-0.2.0-cp36-none-any.whl size=19884 sha256=de314085a69f1942014815306e49e2f2a0d9d98330e73867bdc68eea6a434489
  Stored in directory: /tmp/pip-ephem-wheel-cache-cpvzjcm6/wheels/3d/53/af/92e56f83db191b0579d21e8385d61a92a502e66443b23c7e16
Successfully built microdf
  Cloning https://github.com/MaxGhenis/scf.git to /tmp/pip-req-build-2bdl31ez
  Running command git clone -q https://github.com/MaxGhenis/scf.git /tmp/pip-req-build-2bdl31ez
  Created wheel for scf: filename=scf-0.1.0-cp36-none-any.whl size=2189 sha256=f6d0dd8eb2aa8febf6c52fb0ee1667c112f65d3e1269b2930e0a82db972b10c5
  Stored in directory: /tmp/pip-ephem-wheel-cache-nfymt6qv/wheels/cf/20/08/ce527280314aaf7b1b7dc4c9175b23bd9e8e0120187170c7ac
Successfully built scf


In [2]:
# Import libraries
import pandas as pd
import numpy as np
import microdf as mdf
import scf

In [3]:
scf = mdf.read_stata_zip('https://www.federalreserve.gov/econres/files/scfp2019s.zip')
detail = mdf.read_stata_zip('https://www.federalreserve.gov/econres/files/scf2019s.zip')

In [4]:
scf['numextra'] = (detail.x6533 == 1) + (detail.x6534 == 1)

/usr/local/lib/python3.6/dist-packages/pandas/core/computation/expressions.py:204: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  f"evaluating in Python space because the {repr(op_str)} "


In [5]:
scf['numper'] = 1 + scf.famstruct.isin([4, 5]) + scf.kids + scf.numextra

In [6]:
scf['income_pp'] = scf.income / scf.numper
scf['networth_pp'] = scf.networth / scf.numper
# Weight households by amount of people
scf['wgt_numper'] = scf.wgt * scf.numper

In [7]:
# Income quintiles
mdf.add_weighted_quantiles(scf, 'income_pp', 'wgt_numper')

scf.drop(['income_pp_percentile', 'income_pp_2percentile',
           'income_pp_ventile', 'income_pp_decile',
           'income_pp_quartile'],
          axis=1, inplace=True)

scf.groupby('income_pp_quintile')[['wgt_numper']].sum()

,wgt_numper
income_pp_quintile,
1,6.098401e+07
2,6.100444e+07
3,6.099144e+07
4,6.099438e+07
5,6.101400e+07


In [8]:
# Networth quintiles
mdf.add_weighted_quantiles(scf, 'networth_pp', 'wgt_numper')

scf.drop(['networth_pp_percentile', 'networth_pp_2percentile',
           'networth_pp_ventile', 'networth_pp_decile',
           'networth_pp_quartile'],
          axis=1, inplace=True)

# combine 0 and 1
scf['networth_pp_quintile2'] = np.where(scf.networth_pp_quintile < 2, 0, scf.networth_pp_quintile)
scf.groupby('networth_pp_quintile2')[['wgt_numper']].sum()

,wgt_numper
networth_pp_quintile2,
0,6.099631e+07
2,6.099471e+07
3,6.098782e+07
4,6.099198e+07
5,6.101745e+07


In [9]:
# Maybe rename?  '100th doesn't seem right way to name that
scf['networth_pp_quintile2'] = scf['networth_pp_quintile2'].astype(str)
scf['networth_pp_quintile2'].replace({'0': '0-20', '2': '20-40', '3': '40-60',
                                   '4': '60-80', '5': '80-100'},inplace=True)

scf['income_pp_quintile'] = scf['income_pp_quintile'].astype(str)
scf['income_pp_quintile'].replace({'1': '0-20', '2': '20-40', '3': '40-60',
                                   '4': '60-80', '5': '80-100'},inplace=True)

In [10]:
# Identify race codes
scf['race'] = scf['race'].astype(str)
scf['race'].replace({'1': 'White', '2':'Black',
                     '3': 'Hispanic', '5': 'Other'},inplace=True)

# Identify age codes
scf['agecl'] = scf['agecl'].astype(str)
scf['agecl'].replace({'1':'<35', '2': '35-44', '3': '45-54', '4':'55-64',
                      '5':'65-74', '6':'>=75'},inplace=True)

# Identify education level
scf['edcl'] = scf['edcl'].astype(str)
scf['edcl'].replace({'1': 'Less than high school diploma',
                     '2':'High school diploma', '3': 'Some college',
                     '4': 'College graduate'},inplace=True)

In [11]:
# Calculate estimated population
population = (scf.numper * scf.wgt).sum()
population / 1e6

304.9882663276286

In [12]:
scf['has_ed_debt'] = scf.edn_inst > 0
scf['ed_debt_pp'] = scf.edn_inst / scf.numper

In [13]:
scf['opm_pov_thresh'] = (np.where(scf.numper == 1, 12_490, 
                        np.where(scf.numper == 2, 16_910,
                        np.where(scf.numper == 3, 21_330,
                        np.where(scf.numper == 4, 25_750,
                        np.where(scf.numper == 5, 30_170,
                        np.where(scf.numper == 6, 34_590,
                        np.where(scf.numper == 7, 39_010, 43_430))))))))

In [14]:
scf['original_poor'] = scf.income < scf.opm_pov_thresh
total_poor = (scf.wgt_numper * scf.original_poor).sum()
percent_poor = (total_poor / population * 100).round(1)
percent_poor

10.0

## Who holds student debt?

In [15]:
# Calculate total education debt 
total_student_debt = (scf.wgt * scf.edn_inst).sum()
total_student_debt / 1e12

1.1146904950337784

In [16]:
all = pd.DataFrame([mdf.weighted_mean(scf, 'ed_debt_pp', 'wgt_numper'),
                     mdf.weighted_sum(scf, 'has_ed_debt', 'wgt_numper'),
                     mdf.weighted_sum(scf, 'wgt_numper'),
                     mdf.weighted_sum(scf, 'ed_debt_pp', 'wgt_numper')],
                     index=['mean_debt', 'has_debt', 'population', 'total_debt_held'])

all = all.transpose()
all['percent_has_debt'] = (all.has_debt / all.population * 100).round(1)
all['percent_of_total_debt'] = all.total_debt_held / total_student_debt
all['percent_pop'] = all.population / population
all['ratio'] = (all.percent_of_total_debt / all.percent_pop).round(2)
all['mean_debt'] = all['mean_debt'].astype(int)
all = all.reset_index()

In [17]:
# By race
race = scf.groupby('race').apply(lambda x: pd.Series(
    [mdf.weighted_mean(x, 'ed_debt_pp', 'wgt_numper'),
     mdf.weighted_sum(x, 'has_ed_debt', 'wgt_numper'),
     mdf.weighted_sum(x, 'wgt_numper'),
     mdf.weighted_sum(x, 'ed_debt_pp', 'wgt_numper')],
    index=['mean_debt', 'has_debt', 'population', 'total_debt_held']))

race['percent_has_debt'] = (race.has_debt / race.population * 100).round(1)
race['percent_of_total_debt'] = race.total_debt_held / total_student_debt
race['percent_pop'] = race.population / population
race['ratio'] = (race.percent_of_total_debt / race.percent_pop).round(2)
race['mean_debt'] = race['mean_debt'].astype(int)

race = race.reset_index()

In [18]:
# By Education Level
education = scf.groupby('edcl').apply(lambda x: pd.Series(
    [mdf.weighted_mean(x, 'ed_debt_pp', 'wgt_numper'),
     mdf.weighted_sum(x, 'has_ed_debt', 'wgt_numper'),
     mdf.weighted_sum(x, 'wgt_numper'),
     mdf.weighted_sum(x, 'ed_debt_pp', 'wgt_numper')], 
    index=['mean_debt', 'has_debt', 'population', 'total_debt_held']))

education['percent_has_debt'] = (education.has_debt / education.population * 100).round(1)
education['percent_of_total_debt'] = education.total_debt_held / total_student_debt
education['percent_pop'] = education.population / population
education['ratio'] = (education.percent_of_total_debt / education.percent_pop).round(2)
education['mean_debt'] = education['mean_debt'].astype(int)
education = education.reset_index()

In [19]:
# By age Level
age = scf.groupby('agecl').apply(lambda x: pd.Series(
    [mdf.weighted_mean(x, 'ed_debt_pp', 'wgt_numper'),
     mdf.weighted_sum(x, 'has_ed_debt', 'wgt_numper'),
     mdf.weighted_sum(x, 'wgt_numper'),
     mdf.weighted_sum(x, 'ed_debt_pp', 'wgt_numper')], 
    index=['mean_debt', 'has_debt', 'population', 'total_debt_held']))

age['percent_has_debt'] = (age.has_debt / age.population * 100).round(1)
age['percent_of_total_debt'] = age.total_debt_held / total_student_debt
age['percent_pop'] = age.population / population
age['ratio'] = (age.percent_of_total_debt / age.percent_pop).round(2)
age['mean_debt'] = age['mean_debt'].astype(int)
age = age.reset_index()

In [20]:
# By income Level
income = scf.groupby('income_pp_quintile').apply(lambda x: pd.Series(
    [mdf.weighted_mean(x, 'ed_debt_pp', 'wgt_numper'),
     mdf.weighted_sum(x, 'has_ed_debt', 'wgt_numper'),
     mdf.weighted_sum(x, 'wgt_numper'),
     mdf.weighted_sum(x, 'ed_debt_pp', 'wgt_numper')], 
    index=['mean_debt', 'has_debt', 'population', 'total_debt_held']))

income['percent_has_debt'] = (income.has_debt / income.population * 100).round(1)
income['percent_of_total_debt'] = income.total_debt_held / total_student_debt
income['percent_pop'] = income.population / population
income['ratio'] = (income.percent_of_total_debt / income.percent_pop).round(2)
income['mean_debt'] = income['mean_debt'].astype(int)
income = income.reset_index()

In [21]:
# By networth Level
networth = scf.groupby('networth_pp_quintile2').apply(lambda x: pd.Series(
    [mdf.weighted_mean(x, 'ed_debt_pp', 'wgt_numper'),
     mdf.weighted_sum(x, 'has_ed_debt', 'wgt_numper'),
     mdf.weighted_sum(x, 'wgt_numper'),
     mdf.weighted_sum(x, 'ed_debt_pp', 'wgt_numper')], 
    index=['mean_debt', 'has_debt', 'population', 'total_debt_held']))

networth['percent_has_debt'] = (networth.has_debt / networth.population * 100).round(1)
networth['percent_of_total_debt'] = networth.total_debt_held / total_student_debt
networth['percent_pop'] = networth.population / population
networth['ratio'] = (networth.percent_of_total_debt / networth.percent_pop).round(2)
networth['mean_debt'] = networth['mean_debt'].astype(int)
networth = networth.reset_index()

In [22]:
# By poor Level
poor = scf.groupby('original_poor').apply(lambda x: pd.Series(
    [mdf.weighted_mean(x, 'ed_debt_pp', 'wgt_numper'),
     mdf.weighted_sum(x, 'has_ed_debt', 'wgt_numper'),
     mdf.weighted_sum(x, 'wgt_numper'),
     mdf.weighted_sum(x, 'ed_debt_pp', 'wgt_numper')], 
    index=['mean_debt', 'has_debt', 'population', 'total_debt_held']))

poor['percent_has_debt'] = (poor.has_debt / poor.population * 100).round(1)
poor['percent_of_total_debt'] = poor.total_debt_held / total_student_debt
poor['percent_pop'] = poor.population / population
poor['ratio'] = (poor.percent_of_total_debt / poor.percent_pop).round(2)
poor['mean_debt'] = poor['mean_debt'].astype(int)
poor = poor.reset_index()

## Calculate

In [23]:
# Calculate new networth with eduction debt cancellation
scf['new_networth_no_ed_debt'] = scf.networth + scf.edn_inst

In [24]:
# Calculate the ubi per person
ubi = total_student_debt / population
ubi

3654.863541000428

In [25]:
# Give each household their total UBI
scf['total_ubi'] = ubi * scf.numper

In [26]:
# Calculate new networth with UBI
scf['new_networth_ubi'] = scf.total_ubi + scf.networth

In [27]:
# Calculate per person
scf['income_pp'] = scf.income / scf.numper
scf['networth_pp'] = scf.networth / scf.numper

scf['ubi_networth_pp'] = scf.networth_pp + ubi
scf['no_debt_networth_pp'] = scf.networth_pp + scf.ed_debt_pp
scf['ubi_income_pp'] = scf.income_pp + ubi
scf['no_debt_income_pp'] = scf.income_pp + scf.ed_debt_pp

In [28]:
# Check to make sure reforms cost the same
((scf.new_networth_no_ed_debt * scf.wgt).sum()) - ((scf.new_networth_ubi * scf.wgt).sum()) 

0.0

In [29]:
scf['over_50'] = scf.edn_inst - 50_000
scf['over_50'] = np.where(scf.over_50 > 0, scf.over_50, 0)
scf['difference_50'] = scf.edn_inst - scf.over_50
scf['ed_debt_pp_50'] = scf.difference_50 / scf.numper

In [30]:
total_over_50 = (scf.over_50 * scf.wgt).sum()
total_over_50 / 1e9

401.7215621860863

In [31]:
# Calculate new networth with eduction debt cancellation 50k
scf['new_networth_no_ed_debt_50'] = scf.networth + scf.difference_50

In [32]:
# Calculate the UBI per person
total_under_50 = total_student_debt - total_over_50
ubi_50 = total_under_50 / population
ubi_50

2337.692992037264

In [33]:
# Give each household their total  50k
scf['total_ubi_50'] = ubi_50 * scf.numper

In [34]:
# Calculate new networth with UBI 50k
scf['new_networth_ubi_50'] = scf.total_ubi_50 + scf.networth

In [35]:
scf['ubi_networth_pp_50'] = scf.networth_pp + ubi_50
scf['no_debt_networth_pp_50'] = scf.networth_pp + scf.ed_debt_pp_50
scf['ubi_income_pp_50'] = scf.income_pp + ubi_50
scf['no_debt_income_pp_50'] = scf.income_pp + scf.ed_debt_pp_50

In [36]:
# Check to make sure reforms cost the same
((scf.new_networth_no_ed_debt_50 * scf.wgt).sum()) - ((scf.new_networth_ubi_50 * scf.wgt).sum()) 

0.0

In [37]:
scf['over_10'] = scf.edn_inst - 10_000
scf['over_10'] = np.where(scf.over_10 > 0, scf.over_10, 0)
scf['difference_10'] = scf.edn_inst - scf.over_10
scf['ed_debt_pp_10'] = scf.difference_10 / scf.numper

In [38]:
total_over_10 = (scf.over_10 * scf.wgt).sum()
total_over_10 / 1e9

879.4291895236232

In [39]:
# Calculate new networth with eduction debt cancellation 10k
scf['new_networth_no_ed_debt_10'] = scf.networth + scf.difference_10

In [40]:
# Calculate the UBI per person
total_under_10 = total_student_debt - total_over_10
ubi_10 = total_under_10 / population
ubi_10

771.3782183916203

In [41]:
# Give each household their total  10k
scf['total_ubi_10'] = ubi_10 * scf.numper

In [42]:
# Calculate new networth with UBI 50k
scf['new_networth_ubi_10'] = scf.total_ubi_10 + scf.networth

In [43]:
scf['ubi_networth_pp_10'] = scf.networth_pp + ubi_10
scf['no_debt_networth_pp_10'] = scf.networth_pp + scf.ed_debt_pp_10
scf['ubi_income_pp_10'] = scf.income_pp + ubi_10
scf['no_debt_income_pp_10'] = scf.income_pp + scf.ed_debt_pp_10

In [44]:
# Check to make sure reforms cost the same
((scf.new_networth_no_ed_debt_10 * scf.wgt).sum()) - ((scf.new_networth_ubi_10 * scf.wgt).sum()) 

0.0

## 50 Groupings

In [45]:
def all_fun_50(index_num):
  target_persons = scf.copy(deep=True)

  # Calculate average benefit by reform
  start = mdf.weighted_mean(target_persons, 'networth_pp', 'wgt_numper')
  ed_debt = mdf.weighted_mean(target_persons, 'no_debt_networth_pp_50', 'wgt_numper')
  ubi = mdf.weighted_mean(target_persons, 'ubi_networth_pp_50', 'wgt_numper')

  ed_debt_change_50 = ed_debt - start
  ubi_change_50 = ubi - start

  # Calculate percent better off with ed debt cancellation
  target_persons['better_with_ed_50'] = target_persons.ubi_networth_pp_50 < target_persons.no_debt_networth_pp_50
  total_better_off_with_ed_50 = (target_persons.better_with_ed_50 * target_persons.wgt_numper).sum()
  target_pop = target_persons.wgt_numper.sum()
  percent_better_off_with_ed_50 = (total_better_off_with_ed_50 / target_pop * 100).round(1)

  # Calculate percent out of debt
  target_persons['initial_debt_50'] = target_persons.networth < 0
  target_persons['ubi_debt_50'] = target_persons.ubi_networth_pp < 0
  target_persons['no_ed_debt_debt_50'] = target_persons.no_debt_networth_pp < 0

  percent_initial_debt_50 = (((target_persons.initial_debt_50 
                           * target_persons.wgt_numper).sum()) / target_pop 
                          * 100).round(1)
  percent_ubi_debt_50 = (((target_persons.ubi_debt_50 
                           * target_persons.wgt_numper).sum()) / target_pop
                      * 100).round(1)
  percent_no_ed_debt_debt_50 = (((target_persons.no_ed_debt_debt_50 
                           * target_persons.wgt_numper).sum()) / target_pop
                           * 100).round(1)

  # Calculate percent poor
  total_original_poor_50 = (target_persons.original_poor * target_persons.wgt_numper).sum()
  percent_original_poor_50 = (total_original_poor_50 / target_pop * 100).round(1)

  target_persons['ubi_income_50'] = target_persons.total_ubi_50 + target_persons.income
  target_persons['ubi_poor_50'] = target_persons.ubi_income_50 < target_persons.opm_pov_thresh
  ubi_total_poor_50 = (target_persons.wgt_numper * target_persons.ubi_poor_50).sum()
  ubi_percent_poor_50 = ubi_total_poor_50 / target_pop
  percent_ubi_poor_50 = (ubi_percent_poor_50 * 100).round(1)

  target_persons['ed_debt_income_50'] = target_persons.difference_50 + target_persons.income
  target_persons['ed_debt_poor_50'] = target_persons.ed_debt_income_50 < target_persons.opm_pov_thresh
  ed_debt_total_poor_50 = (target_persons.wgt_numper * target_persons.ed_debt_poor_50).sum()
  ed_debt_percent_poor_50 = ed_debt_total_poor_50 / target_pop
  percent_ed_debt_poor_50 = (ed_debt_percent_poor_50 * 100).round(1)

  # Calculate poverty gap
  target_persons['ubi_poverty_gap_50'] = target_persons.opm_pov_thresh - target_persons.ubi_income_50
  ubi_gap_billions_50 = (((target_persons.ubi_poor_50 * target_persons.ubi_poverty_gap_50 * target_persons.wgt_numper).sum()) / 1e9).round(1)

  target_persons['ed_debt_poverty_gap_50'] = target_persons.opm_pov_thresh - target_persons.ed_debt_income_50
  ed_debt_gap_billions_50 = (((target_persons.ed_debt_poor_50 * target_persons.ed_debt_poverty_gap_50 * target_persons.wgt_numper).sum()) / 1e9).round(1)

  # Convert and round
  ed_debt_change_50 = ed_debt_change_50.astype(int)
  ubi_change_50 = ubi_change_50.astype(int)

  return pd.Series([ed_debt_change_50, ubi_change_50, percent_better_off_with_ed_50, 
                    percent_initial_debt_50, percent_ubi_debt_50, 
                    percent_no_ed_debt_debt_50, percent_ubi_poor_50, percent_ed_debt_poor_50,
                    ubi_gap_billions_50, ed_debt_gap_billions_50])

In [46]:
def all_row_50(row):
  return all_fun_50(row.index)

In [47]:
all[['ed_debt_change_50', 'ubi_change_50', 'percent_better_off_with_ed_50',
        'in_debt_50', 'in_debt_ubi_50', 'in_debt_no_ed_debt_50',
      'percent_ubi_poor_50', 'percent_ed_debt_poor_50', 
     'ubi_gap_billions_50', 'ed_debt_gap_billions_50']] = all.apply(all_row_50, axis=1)

In [48]:
def race_fun_50(race):
  target_persons = scf[scf.race==race].copy(deep=True)

  # Calculate average benefit by reform
  start = mdf.weighted_mean(target_persons, 'networth_pp', 'wgt_numper')
  ed_debt = mdf.weighted_mean(target_persons, 'no_debt_networth_pp_50', 'wgt_numper')
  ubi = mdf.weighted_mean(target_persons, 'ubi_networth_pp_50', 'wgt_numper')

  ed_debt_change_50 = ed_debt - start
  ubi_change_50 = ubi - start

  # Calculate percent better off with ed debt cancellation
  target_persons['better_with_ed_50'] = target_persons.ubi_networth_pp_50 < target_persons.no_debt_networth_pp_50
  total_better_off_with_ed_50 = (target_persons.better_with_ed_50 * target_persons.wgt_numper).sum()
  target_pop = target_persons.wgt_numper.sum()
  percent_better_off_with_ed_50 = (total_better_off_with_ed_50 / target_pop * 100).round(1)

  # Calculate percent out of debt
  target_persons['initial_debt_50'] = target_persons.networth < 0
  target_persons['ubi_debt_50'] = target_persons.ubi_networth_pp < 0
  target_persons['no_ed_debt_debt_50'] = target_persons.no_debt_networth_pp < 0

  percent_initial_debt_50 = (((target_persons.initial_debt_50 
                           * target_persons.wgt_numper).sum()) / target_pop 
                          * 100).round(1)
  percent_ubi_debt_50 = (((target_persons.ubi_debt_50 
                           * target_persons.wgt_numper).sum()) / target_pop
                      * 100).round(1)
  percent_no_ed_debt_debt_50 = (((target_persons.no_ed_debt_debt_50 
                           * target_persons.wgt_numper).sum()) / target_pop
                           * 100).round(1)

  # Calculate percent poor
  total_original_poor_50 = (target_persons.original_poor * target_persons.wgt_numper).sum()
  percent_original_poor_50 = (total_original_poor_50 / target_pop * 100).round(1)

  target_persons['ubi_income_50'] = target_persons.total_ubi_50 + target_persons.income
  target_persons['ubi_poor_50'] = target_persons.ubi_income_50 < target_persons.opm_pov_thresh
  ubi_total_poor_50 = (target_persons.wgt_numper * target_persons.ubi_poor_50).sum()
  ubi_percent_poor_50 = ubi_total_poor_50 / target_pop
  percent_ubi_poor_50 = (ubi_percent_poor_50 * 100).round(1)

  target_persons['ed_debt_income_50'] = target_persons.difference_50 + target_persons.income
  target_persons['ed_debt_poor_50'] = target_persons.ed_debt_income_50 < target_persons.opm_pov_thresh
  ed_debt_total_poor_50 = (target_persons.wgt_numper * target_persons.ed_debt_poor_50).sum()
  ed_debt_percent_poor_50 = ed_debt_total_poor_50 / target_pop
  percent_ed_debt_poor_50 = (ed_debt_percent_poor_50 * 100).round(1)

  # Calculate poverty gap
  target_persons['ubi_poverty_gap_50'] = target_persons.opm_pov_thresh - target_persons.ubi_income_50
  ubi_gap_billions_50 = (((target_persons.ubi_poor_50 * target_persons.ubi_poverty_gap_50 * target_persons.wgt_numper).sum()) / 1e9).round(1)

  target_persons['ed_debt_poverty_gap_50'] = target_persons.opm_pov_thresh - target_persons.ed_debt_income_50
  ed_debt_gap_billions_50 = (((target_persons.ed_debt_poor_50 * target_persons.ed_debt_poverty_gap_50 * target_persons.wgt_numper).sum()) / 1e9).round(1)

  # Convert and round
  ed_debt_change_50 = ed_debt_change_50.astype(int)
  ubi_change_50 = ubi_change_50.astype(int)

  return pd.Series([ed_debt_change_50, ubi_change_50, percent_better_off_with_ed_50, 
                    percent_initial_debt_50, percent_ubi_debt_50, 
                    percent_no_ed_debt_debt_50, percent_ubi_poor_50, percent_ed_debt_poor_50,
                    ubi_gap_billions_50, ed_debt_gap_billions_50])

In [49]:
def race_row_50(row):
  return race_fun_50(row.race)

In [50]:
race[['ed_debt_change_50', 'ubi_change_50', 'percent_better_off_with_ed_50',
        'in_debt_50', 'in_debt_ubi_50', 'in_debt_no_ed_debt_50',
      'percent_ubi_poor_50', 'percent_ed_debt_poor_50', 
     'ubi_gap_billions_50', 'ed_debt_gap_billions_50']] = race.apply(race_row_50, axis=1)

In [51]:
def education_fun_50(edcl):
  target_persons = scf[scf.edcl==edcl].copy(deep=True)

  # Calculate average benefit by reform
  start = mdf.weighted_mean(target_persons, 'networth_pp', 'wgt_numper')
  ed_debt = mdf.weighted_mean(target_persons, 'no_debt_networth_pp_50', 'wgt_numper')
  ubi = mdf.weighted_mean(target_persons, 'ubi_networth_pp_50', 'wgt_numper')

  ed_debt_change_50 = ed_debt - start
  ubi_change_50 = ubi - start

  # Calculate percent better off with ed debt cancellation
  target_persons['better_with_ed_50'] = target_persons.ubi_networth_pp_50 < target_persons.no_debt_networth_pp_50
  total_better_off_with_ed_50 = (target_persons.better_with_ed_50 * target_persons.wgt_numper).sum()
  target_pop = target_persons.wgt_numper.sum()
  percent_better_off_with_ed_50 = (total_better_off_with_ed_50 / target_pop * 100).round(1)

  # Calculate percent out of debt
  target_persons['initial_debt_50'] = target_persons.networth < 0
  target_persons['ubi_debt_50'] = target_persons.ubi_networth_pp < 0
  target_persons['no_ed_debt_debt_50'] = target_persons.no_debt_networth_pp < 0

  percent_initial_debt_50 = (((target_persons.initial_debt_50 
                           * target_persons.wgt_numper).sum()) / target_pop 
                          * 100).round(1)
  percent_ubi_debt_50 = (((target_persons.ubi_debt_50 
                           * target_persons.wgt_numper).sum()) / target_pop
                      * 100).round(1)
  percent_no_ed_debt_debt_50 = (((target_persons.no_ed_debt_debt_50 
                           * target_persons.wgt_numper).sum()) / target_pop
                           * 100).round(1)

  # Calculate percent poor
  total_original_poor_50 = (target_persons.original_poor * target_persons.wgt_numper).sum()
  percent_original_poor_50 = (total_original_poor_50 / target_pop * 100).round(1)

  target_persons['ubi_income_50'] = target_persons.total_ubi_50 + target_persons.income
  target_persons['ubi_poor_50'] = target_persons.ubi_income_50 < target_persons.opm_pov_thresh
  ubi_total_poor_50 = (target_persons.wgt_numper * target_persons.ubi_poor_50).sum()
  ubi_percent_poor_50 = ubi_total_poor_50 / target_pop
  percent_ubi_poor_50 = (ubi_percent_poor_50 * 100).round(1)

  target_persons['ed_debt_income_50'] = target_persons.difference_50 + target_persons.income
  target_persons['ed_debt_poor_50'] = target_persons.ed_debt_income_50 < target_persons.opm_pov_thresh
  ed_debt_total_poor_50 = (target_persons.wgt_numper * target_persons.ed_debt_poor_50).sum()
  ed_debt_percent_poor_50 = ed_debt_total_poor_50 / target_pop
  percent_ed_debt_poor_50 = (ed_debt_percent_poor_50 * 100).round(1)

  # Calculate poverty gap
  target_persons['ubi_poverty_gap_50'] = target_persons.opm_pov_thresh - target_persons.ubi_income_50
  ubi_gap_billions_50 = (((target_persons.ubi_poor_50 * target_persons.ubi_poverty_gap_50 * target_persons.wgt_numper).sum()) / 1e9).round(1)

  target_persons['ed_debt_poverty_gap_50'] = target_persons.opm_pov_thresh - target_persons.ed_debt_income_50
  ed_debt_gap_billions_50 = (((target_persons.ed_debt_poor_50 * target_persons.ed_debt_poverty_gap_50 * target_persons.wgt_numper).sum()) / 1e9).round(1)

  # Convert and round
  ed_debt_change_50 = ed_debt_change_50.astype(int)
  ubi_change_50 = ubi_change_50.astype(int)

  return pd.Series([ed_debt_change_50, ubi_change_50, percent_better_off_with_ed_50, 
                    percent_initial_debt_50, percent_ubi_debt_50, 
                    percent_no_ed_debt_debt_50, percent_ubi_poor_50, percent_ed_debt_poor_50,
                    ubi_gap_billions_50, ed_debt_gap_billions_50])

In [52]:
def education_row_50(row):
  return education_fun_50(row.edcl)

In [53]:
education[['ed_debt_change_50', 'ubi_change_50', 'percent_better_off_with_ed_50',
        'in_debt_50', 'in_debt_ubi_50', 'in_debt_no_ed_debt_50',
      'percent_ubi_poor_50', 'percent_ed_debt_poor_50', 
     'ubi_gap_billions_50', 'ed_debt_gap_billions_50']] = education.apply(education_row_50, axis=1)

In [54]:
def age_fun_50(agecl):
  target_persons = scf[scf.agecl==agecl].copy(deep=True)

  # Calculate average benefit by reform
  start = mdf.weighted_mean(target_persons, 'networth_pp', 'wgt_numper')
  ed_debt = mdf.weighted_mean(target_persons, 'no_debt_networth_pp_50', 'wgt_numper')
  ubi = mdf.weighted_mean(target_persons, 'ubi_networth_pp_50', 'wgt_numper')

  ed_debt_change_50 = ed_debt - start
  ubi_change_50 = ubi - start

  # Calculate percent better off with ed debt cancellation
  target_persons['better_with_ed_50'] = target_persons.ubi_networth_pp_50 < target_persons.no_debt_networth_pp_50
  total_better_off_with_ed_50 = (target_persons.better_with_ed_50 * target_persons.wgt_numper).sum()
  target_pop = target_persons.wgt_numper.sum()
  percent_better_off_with_ed_50 = (total_better_off_with_ed_50 / target_pop * 100).round(1)

  # Calculate percent out of debt
  target_persons['initial_debt_50'] = target_persons.networth < 0
  target_persons['ubi_debt_50'] = target_persons.ubi_networth_pp < 0
  target_persons['no_ed_debt_debt_50'] = target_persons.no_debt_networth_pp < 0

  percent_initial_debt_50 = (((target_persons.initial_debt_50 
                           * target_persons.wgt_numper).sum()) / target_pop 
                          * 100).round(1)
  percent_ubi_debt_50 = (((target_persons.ubi_debt_50 
                           * target_persons.wgt_numper).sum()) / target_pop
                      * 100).round(1)
  percent_no_ed_debt_debt_50 = (((target_persons.no_ed_debt_debt_50 
                           * target_persons.wgt_numper).sum()) / target_pop
                           * 100).round(1)

  # Calculate percent poor
  total_original_poor_50 = (target_persons.original_poor * target_persons.wgt_numper).sum()
  percent_original_poor_50 = (total_original_poor_50 / target_pop * 100).round(1)

  target_persons['ubi_income_50'] = target_persons.total_ubi_50 + target_persons.income
  target_persons['ubi_poor_50'] = target_persons.ubi_income_50 < target_persons.opm_pov_thresh
  ubi_total_poor_50 = (target_persons.wgt_numper * target_persons.ubi_poor_50).sum()
  ubi_percent_poor_50 = ubi_total_poor_50 / target_pop
  percent_ubi_poor_50 = (ubi_percent_poor_50 * 100).round(1)

  target_persons['ed_debt_income_50'] = target_persons.difference_50 + target_persons.income
  target_persons['ed_debt_poor_50'] = target_persons.ed_debt_income_50 < target_persons.opm_pov_thresh
  ed_debt_total_poor_50 = (target_persons.wgt_numper * target_persons.ed_debt_poor_50).sum()
  ed_debt_percent_poor_50 = ed_debt_total_poor_50 / target_pop
  percent_ed_debt_poor_50 = (ed_debt_percent_poor_50 * 100).round(1)

  # Calculate poverty gap
  target_persons['ubi_poverty_gap_50'] = target_persons.opm_pov_thresh - target_persons.ubi_income_50
  ubi_gap_billions_50 = (((target_persons.ubi_poor_50 * target_persons.ubi_poverty_gap_50 * target_persons.wgt_numper).sum()) / 1e9).round(1)

  target_persons['ed_debt_poverty_gap_50'] = target_persons.opm_pov_thresh - target_persons.ed_debt_income_50
  ed_debt_gap_billions_50 = (((target_persons.ed_debt_poor_50 * target_persons.ed_debt_poverty_gap_50 * target_persons.wgt_numper).sum()) / 1e9).round(1)

  # Convert and round
  ed_debt_change_50 = ed_debt_change_50.astype(int)
  ubi_change_50 = ubi_change_50.astype(int)

  return pd.Series([ed_debt_change_50, ubi_change_50, percent_better_off_with_ed_50, 
                    percent_initial_debt_50, percent_ubi_debt_50, 
                    percent_no_ed_debt_debt_50, percent_ubi_poor_50, percent_ed_debt_poor_50,
                    ubi_gap_billions_50, ed_debt_gap_billions_50])

In [55]:
def age_row_50(row):
  return age_fun_50(row.agecl)

In [56]:
age[['ed_debt_change_50', 'ubi_change_50', 'percent_better_off_with_ed_50',
        'in_debt_50', 'in_debt_ubi_50', 'in_debt_no_ed_debt_50',
      'percent_ubi_poor_50', 'percent_ed_debt_poor_50', 
     'ubi_gap_billions_50', 'ed_debt_gap_billions_50']] = age.apply(age_row_50, axis=1)

In [57]:
def income_fun_50(income_pp_quintile):
  target_persons = scf[scf.income_pp_quintile==income_pp_quintile].copy(deep=True)

  # Calculate average benefit by reform
  start = mdf.weighted_mean(target_persons, 'networth_pp', 'wgt_numper')
  ed_debt = mdf.weighted_mean(target_persons, 'no_debt_networth_pp_50', 'wgt_numper')
  ubi = mdf.weighted_mean(target_persons, 'ubi_networth_pp_50', 'wgt_numper')

  ed_debt_change_50 = ed_debt - start
  ubi_change_50 = ubi - start

  # Calculate percent better off with ed debt cancellation
  target_persons['better_with_ed_50'] = target_persons.ubi_networth_pp_50 < target_persons.no_debt_networth_pp_50
  total_better_off_with_ed_50 = (target_persons.better_with_ed_50 * target_persons.wgt_numper).sum()
  target_pop = target_persons.wgt_numper.sum()
  percent_better_off_with_ed_50 = (total_better_off_with_ed_50 / target_pop * 100).round(1)

  # Calculate percent out of debt
  target_persons['initial_debt_50'] = target_persons.networth < 0
  target_persons['ubi_debt_50'] = target_persons.ubi_networth_pp < 0
  target_persons['no_ed_debt_debt_50'] = target_persons.no_debt_networth_pp < 0

  percent_initial_debt_50 = (((target_persons.initial_debt_50 
                           * target_persons.wgt_numper).sum()) / target_pop 
                          * 100).round(1)
  percent_ubi_debt_50 = (((target_persons.ubi_debt_50 
                           * target_persons.wgt_numper).sum()) / target_pop
                      * 100).round(1)
  percent_no_ed_debt_debt_50 = (((target_persons.no_ed_debt_debt_50 
                           * target_persons.wgt_numper).sum()) / target_pop
                           * 100).round(1)

  # Calculate percent poor
  total_original_poor_50 = (target_persons.original_poor * target_persons.wgt_numper).sum()
  percent_original_poor_50 = (total_original_poor_50 / target_pop * 100).round(1)

  target_persons['ubi_income_50'] = target_persons.total_ubi_50 + target_persons.income
  target_persons['ubi_poor_50'] = target_persons.ubi_income_50 < target_persons.opm_pov_thresh
  ubi_total_poor_50 = (target_persons.wgt_numper * target_persons.ubi_poor_50).sum()
  ubi_percent_poor_50 = ubi_total_poor_50 / target_pop
  percent_ubi_poor_50 = (ubi_percent_poor_50 * 100).round(1)

  target_persons['ed_debt_income_50'] = target_persons.difference_50 + target_persons.income
  target_persons['ed_debt_poor_50'] = target_persons.ed_debt_income_50 < target_persons.opm_pov_thresh
  ed_debt_total_poor_50 = (target_persons.wgt_numper * target_persons.ed_debt_poor_50).sum()
  ed_debt_percent_poor_50 = ed_debt_total_poor_50 / target_pop
  percent_ed_debt_poor_50 = (ed_debt_percent_poor_50 * 100).round(1)

  # Calculate poverty gap
  target_persons['ubi_poverty_gap_50'] = target_persons.opm_pov_thresh - target_persons.ubi_income_50
  ubi_gap_billions_50 = (((target_persons.ubi_poor_50 * target_persons.ubi_poverty_gap_50 * target_persons.wgt_numper).sum()) / 1e9).round(1)

  target_persons['ed_debt_poverty_gap_50'] = target_persons.opm_pov_thresh - target_persons.ed_debt_income_50
  ed_debt_gap_billions_50 = (((target_persons.ed_debt_poor_50 * target_persons.ed_debt_poverty_gap_50 * target_persons.wgt_numper).sum()) / 1e9).round(1)

  # Convert and round
  ed_debt_change_50 = ed_debt_change_50.astype(int)
  ubi_change_50 = ubi_change_50.astype(int)

  return pd.Series([ed_debt_change_50, ubi_change_50, percent_better_off_with_ed_50, 
                    percent_initial_debt_50, percent_ubi_debt_50, 
                    percent_no_ed_debt_debt_50, percent_ubi_poor_50, percent_ed_debt_poor_50,
                    ubi_gap_billions_50, ed_debt_gap_billions_50])

In [58]:
def income_row_50(row):
  return income_fun_50(row.income_pp_quintile)

In [59]:
income[['ed_debt_change_50', 'ubi_change_50', 'percent_better_off_with_ed_50',
        'in_debt_50', 'in_debt_ubi_50', 'in_debt_no_ed_debt_50',
      'percent_ubi_poor_50', 'percent_ed_debt_poor_50', 
     'ubi_gap_billions_50', 'ed_debt_gap_billions_50']] = income.apply(income_row_50, axis=1)

In [60]:
def networth_fun_50(networth_pp_quintile2):
  target_persons = scf[scf.networth_pp_quintile2==networth_pp_quintile2].copy(deep=True)

  # Calculate average benefit by reform
  start = mdf.weighted_mean(target_persons, 'networth_pp', 'wgt_numper')
  ed_debt = mdf.weighted_mean(target_persons, 'no_debt_networth_pp_50', 'wgt_numper')
  ubi = mdf.weighted_mean(target_persons, 'ubi_networth_pp_50', 'wgt_numper')

  ed_debt_change_50 = ed_debt - start
  ubi_change_50 = ubi - start

  # Calculate percent better off with ed debt cancellation
  target_persons['better_with_ed_50'] = target_persons.ubi_networth_pp_50 < target_persons.no_debt_networth_pp_50
  total_better_off_with_ed_50 = (target_persons.better_with_ed_50 * target_persons.wgt_numper).sum()
  target_pop = target_persons.wgt_numper.sum()
  percent_better_off_with_ed_50 = (total_better_off_with_ed_50 / target_pop * 100).round(1)

  # Calculate percent out of debt
  target_persons['initial_debt_50'] = target_persons.networth < 0
  target_persons['ubi_debt_50'] = target_persons.ubi_networth_pp < 0
  target_persons['no_ed_debt_debt_50'] = target_persons.no_debt_networth_pp < 0

  percent_initial_debt_50 = (((target_persons.initial_debt_50 
                           * target_persons.wgt_numper).sum()) / target_pop 
                          * 100).round(1)
  percent_ubi_debt_50 = (((target_persons.ubi_debt_50 
                           * target_persons.wgt_numper).sum()) / target_pop
                      * 100).round(1)
  percent_no_ed_debt_debt_50 = (((target_persons.no_ed_debt_debt_50 
                           * target_persons.wgt_numper).sum()) / target_pop
                           * 100).round(1)

  # Calculate percent poor
  total_original_poor_50 = (target_persons.original_poor * target_persons.wgt_numper).sum()
  percent_original_poor_50 = (total_original_poor_50 / target_pop * 100).round(1)

  target_persons['ubi_income_50'] = target_persons.total_ubi_50 + target_persons.income
  target_persons['ubi_poor_50'] = target_persons.ubi_income_50 < target_persons.opm_pov_thresh
  ubi_total_poor_50 = (target_persons.wgt_numper * target_persons.ubi_poor_50).sum()
  ubi_percent_poor_50 = ubi_total_poor_50 / target_pop
  percent_ubi_poor_50 = (ubi_percent_poor_50 * 100).round(1)

  target_persons['ed_debt_income_50'] = target_persons.difference_50 + target_persons.income
  target_persons['ed_debt_poor_50'] = target_persons.ed_debt_income_50 < target_persons.opm_pov_thresh
  ed_debt_total_poor_50 = (target_persons.wgt_numper * target_persons.ed_debt_poor_50).sum()
  ed_debt_percent_poor_50 = ed_debt_total_poor_50 / target_pop
  percent_ed_debt_poor_50 = (ed_debt_percent_poor_50 * 100).round(1)

  # Calculate poverty gap
  target_persons['ubi_poverty_gap_50'] = target_persons.opm_pov_thresh - target_persons.ubi_income_50
  ubi_gap_billions_50 = (((target_persons.ubi_poor_50 * target_persons.ubi_poverty_gap_50 * target_persons.wgt_numper).sum()) / 1e9).round(1)

  target_persons['ed_debt_poverty_gap_50'] = target_persons.opm_pov_thresh - target_persons.ed_debt_income_50
  ed_debt_gap_billions_50 = (((target_persons.ed_debt_poor_50 * target_persons.ed_debt_poverty_gap_50 * target_persons.wgt_numper).sum()) / 1e9).round(1)

  # Convert and round
  ed_debt_change_50 = ed_debt_change_50.astype(int)
  ubi_change_50 = ubi_change_50.astype(int)

  return pd.Series([ed_debt_change_50, ubi_change_50, percent_better_off_with_ed_50, 
                    percent_initial_debt_50, percent_ubi_debt_50, 
                    percent_no_ed_debt_debt_50, percent_ubi_poor_50, percent_ed_debt_poor_50,
                    ubi_gap_billions_50, ed_debt_gap_billions_50])

In [61]:
def networth_row_50(row):
  return networth_fun_50(row.networth_pp_quintile2)

In [62]:
networth[['ed_debt_change_50', 'ubi_change_50', 'percent_better_off_with_ed_50',
        'in_debt_50', 'in_debt_ubi_50', 'in_debt_no_ed_debt_50',
      'percent_ubi_poor_50', 'percent_ed_debt_poor_50', 
     'ubi_gap_billions_50', 'ed_debt_gap_billions_50']] = networth.apply(networth_row_50, axis=1)

In [63]:
def poor_fun_50(original_poor):
  target_persons = scf[scf.original_poor==original_poor].copy(deep=True)

  # Calculate average benefit by reform
  start = mdf.weighted_mean(target_persons, 'networth_pp', 'wgt_numper')
  ed_debt = mdf.weighted_mean(target_persons, 'no_debt_networth_pp_50', 'wgt_numper')
  ubi = mdf.weighted_mean(target_persons, 'ubi_networth_pp_50', 'wgt_numper')

  ed_debt_change_50 = ed_debt - start
  ubi_change_50 = ubi - start

  # Calculate percent better off with ed debt cancellation
  target_persons['better_with_ed_50'] = target_persons.ubi_networth_pp_50 < target_persons.no_debt_networth_pp_50
  total_better_off_with_ed_50 = (target_persons.better_with_ed_50 * target_persons.wgt_numper).sum()
  target_pop = target_persons.wgt_numper.sum()
  percent_better_off_with_ed_50 = (total_better_off_with_ed_50 / target_pop * 100).round(1)

  # Calculate percent out of debt
  target_persons['initial_debt_50'] = target_persons.networth < 0
  target_persons['ubi_debt_50'] = target_persons.ubi_networth_pp < 0
  target_persons['no_ed_debt_debt_50'] = target_persons.no_debt_networth_pp < 0

  percent_initial_debt_50 = (((target_persons.initial_debt_50 
                           * target_persons.wgt_numper).sum()) / target_pop 
                          * 100).round(1)
  percent_ubi_debt_50 = (((target_persons.ubi_debt_50 
                           * target_persons.wgt_numper).sum()) / target_pop
                      * 100).round(1)
  percent_no_ed_debt_debt_50 = (((target_persons.no_ed_debt_debt_50 
                           * target_persons.wgt_numper).sum()) / target_pop
                           * 100).round(1)

  # Calculate percent poor
  total_original_poor_50 = (target_persons.original_poor * target_persons.wgt_numper).sum()
  percent_original_poor_50 = (total_original_poor_50 / target_pop * 100).round(1)

  target_persons['ubi_income_50'] = target_persons.total_ubi_50 + target_persons.income
  target_persons['ubi_poor_50'] = target_persons.ubi_income_50 < target_persons.opm_pov_thresh
  ubi_total_poor_50 = (target_persons.wgt_numper * target_persons.ubi_poor_50).sum()
  ubi_percent_poor_50 = ubi_total_poor_50 / target_pop
  percent_ubi_poor_50 = (ubi_percent_poor_50 * 100).round(1)

  target_persons['ed_debt_income_50'] = target_persons.difference_50 + target_persons.income
  target_persons['ed_debt_poor_50'] = target_persons.ed_debt_income_50 < target_persons.opm_pov_thresh
  ed_debt_total_poor_50 = (target_persons.wgt_numper * target_persons.ed_debt_poor_50).sum()
  ed_debt_percent_poor_50 = ed_debt_total_poor_50 / target_pop
  percent_ed_debt_poor_50 = (ed_debt_percent_poor_50 * 100).round(1)

  # Calculate poverty gap
  target_persons['ubi_poverty_gap_50'] = target_persons.opm_pov_thresh - target_persons.ubi_income_50
  ubi_gap_billions_50 = (((target_persons.ubi_poor_50 * target_persons.ubi_poverty_gap_50 * target_persons.wgt_numper).sum()) / 1e9).round(1)

  target_persons['ed_debt_poverty_gap_50'] = target_persons.opm_pov_thresh - target_persons.ed_debt_income_50
  ed_debt_gap_billions_50 = (((target_persons.ed_debt_poor_50 * target_persons.ed_debt_poverty_gap_50 * target_persons.wgt_numper).sum()) / 1e9).round(1)

  # Convert and round
  ed_debt_change_50 = ed_debt_change_50.astype(int)
  ubi_change_50 = ubi_change_50.astype(int)

  return pd.Series([ed_debt_change_50, ubi_change_50, percent_better_off_with_ed_50, 
                    percent_initial_debt_50, percent_ubi_debt_50, 
                    percent_no_ed_debt_debt_50, percent_ubi_poor_50, percent_ed_debt_poor_50,
                    ubi_gap_billions_50, ed_debt_gap_billions_50])

In [64]:
def poor_row_50(row):
  return poor_fun_50(row.original_poor)

In [65]:
poor[['ed_debt_change_50', 'ubi_change_50', 'percent_better_off_with_ed_50',
        'in_debt_50', 'in_debt_ubi_50', 'in_debt_no_ed_debt_50',
      'percent_ubi_poor_50', 'percent_ed_debt_poor_50', 
     'ubi_gap_billions_50', 'ed_debt_gap_billions_50']] = poor.apply(poor_row_50, axis=1)

## 10 Groupings

In [66]:
def all_fun_10(index_num):
  target_persons = scf.copy(deep=True)

  # Calculate average benefit by reform
  start = mdf.weighted_mean(target_persons, 'networth_pp', 'wgt_numper')
  ed_debt = mdf.weighted_mean(target_persons, 'no_debt_networth_pp_10', 'wgt_numper')
  ubi = mdf.weighted_mean(target_persons, 'ubi_networth_pp_10', 'wgt_numper')

  ed_debt_change_10 = ed_debt - start
  ubi_change_10 = ubi - start

  # Calculate percent better off with ed debt cancellation
  target_persons['better_with_ed_10'] = target_persons.ubi_networth_pp_10 < target_persons.no_debt_networth_pp_10
  total_better_off_with_ed_10 = (target_persons.better_with_ed_10 * target_persons.wgt_numper).sum()
  target_pop = target_persons.wgt_numper.sum()
  percent_better_off_with_ed_10 = (total_better_off_with_ed_10 / target_pop * 100).round(1)

  # Calculate percent out of debt
  target_persons['initial_debt_10'] = target_persons.networth < 0
  target_persons['ubi_debt_10'] = target_persons.ubi_networth_pp < 0
  target_persons['no_ed_debt_debt_10'] = target_persons.no_debt_networth_pp < 0

  percent_initial_debt_10 = (((target_persons.initial_debt_10 
                           * target_persons.wgt_numper).sum()) / target_pop 
                          * 100).round(1)
  percent_ubi_debt_10 = (((target_persons.ubi_debt_10 
                           * target_persons.wgt_numper).sum()) / target_pop
                      * 100).round(1)
  percent_no_ed_debt_debt_10 = (((target_persons.no_ed_debt_debt_10 
                           * target_persons.wgt_numper).sum()) / target_pop
                           * 100).round(1)

  # Calculate percent poor
  total_original_poor_10 = (target_persons.original_poor * target_persons.wgt_numper).sum()
  percent_original_poor_10 = (total_original_poor_10 / target_pop * 100).round(1)

  target_persons['ubi_income_10'] = target_persons.total_ubi_10 + target_persons.income
  target_persons['ubi_poor_10'] = target_persons.ubi_income_10 < target_persons.opm_pov_thresh
  ubi_total_poor_10 = (target_persons.wgt_numper * target_persons.ubi_poor_10).sum()
  ubi_percent_poor_10 = ubi_total_poor_10 / target_pop
  percent_ubi_poor_10 = (ubi_percent_poor_10 * 100).round(1)

  target_persons['ed_debt_income_10'] = target_persons.difference_10 + target_persons.income
  target_persons['ed_debt_poor_10'] = target_persons.ed_debt_income_10 < target_persons.opm_pov_thresh
  ed_debt_total_poor_10 = (target_persons.wgt_numper * target_persons.ed_debt_poor_10).sum()
  ed_debt_percent_poor_10 = ed_debt_total_poor_10 / target_pop
  percent_ed_debt_poor_10 = (ed_debt_percent_poor_10 * 100).round(1)

  # Calculate poverty gap
  target_persons['ubi_poverty_gap_10'] = target_persons.opm_pov_thresh - target_persons.ubi_income_10
  ubi_gap_billions_10 = (((target_persons.ubi_poor_10 * target_persons.ubi_poverty_gap_10 * target_persons.wgt_numper).sum()) / 1e9).round(1)

  target_persons['ed_debt_poverty_gap_10'] = target_persons.opm_pov_thresh - target_persons.ed_debt_income_10
  ed_debt_gap_billions_10 = (((target_persons.ed_debt_poor_10 * target_persons.ed_debt_poverty_gap_10 * target_persons.wgt_numper).sum()) / 1e9).round(1)

  # Convert and round
  ed_debt_change_10 = ed_debt_change_10.astype(int)
  ubi_change_10 = ubi_change_10.astype(int)

  return pd.Series([ed_debt_change_10, ubi_change_10, percent_better_off_with_ed_10, 
                    percent_initial_debt_10, percent_ubi_debt_10, 
                    percent_no_ed_debt_debt_10, percent_ubi_poor_10, percent_ed_debt_poor_10,
                    ubi_gap_billions_10, ed_debt_gap_billions_10])

In [67]:
def all_row_10(row):
  return all_fun_10(row.index)

In [68]:
all[['ed_debt_change_10', 'ubi_change_10', 'percent_better_off_with_ed_10',
        'in_debt_10', 'in_debt_ubi_10', 'in_debt_no_ed_debt_10',
      'percent_ubi_poor_10', 'percent_ed_debt_poor_10', 
     'ubi_gap_billions_10', 'ed_debt_gap_billions_10']] = all.apply(all_row_10, axis=1)

In [69]:
def race_fun_10(race):
  target_persons = scf[scf.race==race].copy(deep=True)

  # Calculate average benefit by reform
  start = mdf.weighted_mean(target_persons, 'networth_pp', 'wgt_numper')
  ed_debt = mdf.weighted_mean(target_persons, 'no_debt_networth_pp_10', 'wgt_numper')
  ubi = mdf.weighted_mean(target_persons, 'ubi_networth_pp_10', 'wgt_numper')

  ed_debt_change_10 = ed_debt - start
  ubi_change_10 = ubi - start

  # Calculate percent better off with ed debt cancellation
  target_persons['better_with_ed_10'] = target_persons.ubi_networth_pp_10 < target_persons.no_debt_networth_pp_10
  total_better_off_with_ed_10 = (target_persons.better_with_ed_10 * target_persons.wgt_numper).sum()
  target_pop = target_persons.wgt_numper.sum()
  percent_better_off_with_ed_10 = (total_better_off_with_ed_10 / target_pop * 100).round(1)

  # Calculate percent out of debt
  target_persons['initial_debt_10'] = target_persons.networth < 0
  target_persons['ubi_debt_10'] = target_persons.ubi_networth_pp < 0
  target_persons['no_ed_debt_debt_10'] = target_persons.no_debt_networth_pp < 0

  percent_initial_debt_10 = (((target_persons.initial_debt_10 
                           * target_persons.wgt_numper).sum()) / target_pop 
                          * 100).round(1)
  percent_ubi_debt_10 = (((target_persons.ubi_debt_10 
                           * target_persons.wgt_numper).sum()) / target_pop
                      * 100).round(1)
  percent_no_ed_debt_debt_10 = (((target_persons.no_ed_debt_debt_10 
                           * target_persons.wgt_numper).sum()) / target_pop
                           * 100).round(1)

  # Calculate percent poor
  total_original_poor_10 = (target_persons.original_poor * target_persons.wgt_numper).sum()
  percent_original_poor_10 = (total_original_poor_10 / target_pop * 100).round(1)

  target_persons['ubi_income_10'] = target_persons.total_ubi_10 + target_persons.income
  target_persons['ubi_poor_10'] = target_persons.ubi_income_10 < target_persons.opm_pov_thresh
  ubi_total_poor_10 = (target_persons.wgt_numper * target_persons.ubi_poor_10).sum()
  ubi_percent_poor_10 = ubi_total_poor_10 / target_pop
  percent_ubi_poor_10 = (ubi_percent_poor_10 * 100).round(1)

  target_persons['ed_debt_income_10'] = target_persons.difference_10 + target_persons.income
  target_persons['ed_debt_poor_10'] = target_persons.ed_debt_income_10 < target_persons.opm_pov_thresh
  ed_debt_total_poor_10 = (target_persons.wgt_numper * target_persons.ed_debt_poor_10).sum()
  ed_debt_percent_poor_10 = ed_debt_total_poor_10 / target_pop
  percent_ed_debt_poor_10 = (ed_debt_percent_poor_10 * 100).round(1)

  # Calculate poverty gap
  target_persons['ubi_poverty_gap_10'] = target_persons.opm_pov_thresh - target_persons.ubi_income_10
  ubi_gap_billions_10 = (((target_persons.ubi_poor_10 * target_persons.ubi_poverty_gap_10 * target_persons.wgt_numper).sum()) / 1e9).round(1)

  target_persons['ed_debt_poverty_gap_10'] = target_persons.opm_pov_thresh - target_persons.ed_debt_income_10
  ed_debt_gap_billions_10 = (((target_persons.ed_debt_poor_10 * target_persons.ed_debt_poverty_gap_10 * target_persons.wgt_numper).sum()) / 1e9).round(1)

  # Convert and round
  ed_debt_change_10 = ed_debt_change_10.astype(int)
  ubi_change_10 = ubi_change_10.astype(int)

  return pd.Series([ed_debt_change_10, ubi_change_10, percent_better_off_with_ed_10, 
                    percent_initial_debt_10, percent_ubi_debt_10, 
                    percent_no_ed_debt_debt_10, percent_ubi_poor_10, percent_ed_debt_poor_10,
                    ubi_gap_billions_10, ed_debt_gap_billions_10])

In [70]:
def race_row_10(row):
  return race_fun_10(row.race)

In [71]:
race[['ed_debt_change_10', 'ubi_change_10', 'percent_better_off_with_ed_10',
        'in_debt_10', 'in_debt_ubi_10', 'in_debt_no_ed_debt_10',
      'percent_ubi_poor_10', 'percent_ed_debt_poor_10', 
     'ubi_gap_billions_10', 'ed_debt_gap_billions_10']] = race.apply(race_row_10, axis=1)

In [72]:
def education_fun_10(edcl):
  target_persons = scf[scf.edcl==edcl].copy(deep=True)

  # Calculate average benefit by reform
  start = mdf.weighted_mean(target_persons, 'networth_pp', 'wgt_numper')
  ed_debt = mdf.weighted_mean(target_persons, 'no_debt_networth_pp_10', 'wgt_numper')
  ubi = mdf.weighted_mean(target_persons, 'ubi_networth_pp_10', 'wgt_numper')

  ed_debt_change_10 = ed_debt - start
  ubi_change_10 = ubi - start

  # Calculate percent better off with ed debt cancellation
  target_persons['better_with_ed_10'] = target_persons.ubi_networth_pp_10 < target_persons.no_debt_networth_pp_10
  total_better_off_with_ed_10 = (target_persons.better_with_ed_10 * target_persons.wgt_numper).sum()
  target_pop = target_persons.wgt_numper.sum()
  percent_better_off_with_ed_10 = (total_better_off_with_ed_10 / target_pop * 100).round(1)

  # Calculate percent out of debt
  target_persons['initial_debt_10'] = target_persons.networth < 0
  target_persons['ubi_debt_10'] = target_persons.ubi_networth_pp < 0
  target_persons['no_ed_debt_debt_10'] = target_persons.no_debt_networth_pp < 0

  percent_initial_debt_10 = (((target_persons.initial_debt_10 
                           * target_persons.wgt_numper).sum()) / target_pop 
                          * 100).round(1)
  percent_ubi_debt_10 = (((target_persons.ubi_debt_10 
                           * target_persons.wgt_numper).sum()) / target_pop
                      * 100).round(1)
  percent_no_ed_debt_debt_10 = (((target_persons.no_ed_debt_debt_10 
                           * target_persons.wgt_numper).sum()) / target_pop
                           * 100).round(1)

  # Calculate percent poor
  total_original_poor_10 = (target_persons.original_poor * target_persons.wgt_numper).sum()
  percent_original_poor_10 = (total_original_poor_10 / target_pop * 100).round(1)

  target_persons['ubi_income_10'] = target_persons.total_ubi_10 + target_persons.income
  target_persons['ubi_poor_10'] = target_persons.ubi_income_10 < target_persons.opm_pov_thresh
  ubi_total_poor_10 = (target_persons.wgt_numper * target_persons.ubi_poor_10).sum()
  ubi_percent_poor_10 = ubi_total_poor_10 / target_pop
  percent_ubi_poor_10 = (ubi_percent_poor_10 * 100).round(1)

  target_persons['ed_debt_income_10'] = target_persons.difference_10 + target_persons.income
  target_persons['ed_debt_poor_10'] = target_persons.ed_debt_income_10 < target_persons.opm_pov_thresh
  ed_debt_total_poor_10 = (target_persons.wgt_numper * target_persons.ed_debt_poor_10).sum()
  ed_debt_percent_poor_10 = ed_debt_total_poor_10 / target_pop
  percent_ed_debt_poor_10 = (ed_debt_percent_poor_10 * 100).round(1)

  # Calculate poverty gap
  target_persons['ubi_poverty_gap_10'] = target_persons.opm_pov_thresh - target_persons.ubi_income_10
  ubi_gap_billions_10 = (((target_persons.ubi_poor_10 * target_persons.ubi_poverty_gap_10 * target_persons.wgt_numper).sum()) / 1e9).round(1)

  target_persons['ed_debt_poverty_gap_10'] = target_persons.opm_pov_thresh - target_persons.ed_debt_income_10
  ed_debt_gap_billions_10 = (((target_persons.ed_debt_poor_10 * target_persons.ed_debt_poverty_gap_10 * target_persons.wgt_numper).sum()) / 1e9).round(1)

  # Convert and round
  ed_debt_change_10 = ed_debt_change_10.astype(int)
  ubi_change_10 = ubi_change_10.astype(int)

  return pd.Series([ed_debt_change_10, ubi_change_10, percent_better_off_with_ed_10, 
                    percent_initial_debt_10, percent_ubi_debt_10, 
                    percent_no_ed_debt_debt_10, percent_ubi_poor_10, percent_ed_debt_poor_10,
                    ubi_gap_billions_10, ed_debt_gap_billions_10])

In [73]:
def education_row_10(row):
  return education_fun_10(row.edcl)

In [74]:
education[['ed_debt_change_10', 'ubi_change_10', 'percent_better_off_with_ed_10',
        'in_debt_10', 'in_debt_ubi_10', 'in_debt_no_ed_debt_10',
      'percent_ubi_poor_10', 'percent_ed_debt_poor_10', 
     'ubi_gap_billions_10', 'ed_debt_gap_billions_10']] = education.apply(education_row_10, axis=1)

In [75]:
def age_fun_10(agecl):
  target_persons = scf[scf.agecl==agecl].copy(deep=True)

  # Calculate average benefit by reform
  start = mdf.weighted_mean(target_persons, 'networth_pp', 'wgt_numper')
  ed_debt = mdf.weighted_mean(target_persons, 'no_debt_networth_pp_10', 'wgt_numper')
  ubi = mdf.weighted_mean(target_persons, 'ubi_networth_pp_10', 'wgt_numper')

  ed_debt_change_10 = ed_debt - start
  ubi_change_10 = ubi - start

  # Calculate percent better off with ed debt cancellation
  target_persons['better_with_ed_10'] = target_persons.ubi_networth_pp_10 < target_persons.no_debt_networth_pp_10
  total_better_off_with_ed_10 = (target_persons.better_with_ed_10 * target_persons.wgt_numper).sum()
  target_pop = target_persons.wgt_numper.sum()
  percent_better_off_with_ed_10 = (total_better_off_with_ed_10 / target_pop * 100).round(1)

  # Calculate percent out of debt
  target_persons['initial_debt_10'] = target_persons.networth < 0
  target_persons['ubi_debt_10'] = target_persons.ubi_networth_pp < 0
  target_persons['no_ed_debt_debt_10'] = target_persons.no_debt_networth_pp < 0

  percent_initial_debt_10 = (((target_persons.initial_debt_10 
                           * target_persons.wgt_numper).sum()) / target_pop 
                          * 100).round(1)
  percent_ubi_debt_10 = (((target_persons.ubi_debt_10 
                           * target_persons.wgt_numper).sum()) / target_pop
                      * 100).round(1)
  percent_no_ed_debt_debt_10 = (((target_persons.no_ed_debt_debt_10 
                           * target_persons.wgt_numper).sum()) / target_pop
                           * 100).round(1)

  # Calculate percent poor
  total_original_poor_10 = (target_persons.original_poor * target_persons.wgt_numper).sum()
  percent_original_poor_10 = (total_original_poor_10 / target_pop * 100).round(1)

  target_persons['ubi_income_10'] = target_persons.total_ubi_10 + target_persons.income
  target_persons['ubi_poor_10'] = target_persons.ubi_income_10 < target_persons.opm_pov_thresh
  ubi_total_poor_10 = (target_persons.wgt_numper * target_persons.ubi_poor_10).sum()
  ubi_percent_poor_10 = ubi_total_poor_10 / target_pop
  percent_ubi_poor_10 = (ubi_percent_poor_10 * 100).round(1)

  target_persons['ed_debt_income_10'] = target_persons.difference_10 + target_persons.income
  target_persons['ed_debt_poor_10'] = target_persons.ed_debt_income_10 < target_persons.opm_pov_thresh
  ed_debt_total_poor_10 = (target_persons.wgt_numper * target_persons.ed_debt_poor_10).sum()
  ed_debt_percent_poor_10 = ed_debt_total_poor_10 / target_pop
  percent_ed_debt_poor_10 = (ed_debt_percent_poor_10 * 100).round(1)

  # Calculate poverty gap
  target_persons['ubi_poverty_gap_10'] = target_persons.opm_pov_thresh - target_persons.ubi_income_10
  ubi_gap_billions_10 = (((target_persons.ubi_poor_10 * target_persons.ubi_poverty_gap_10 * target_persons.wgt_numper).sum()) / 1e9).round(1)

  target_persons['ed_debt_poverty_gap_10'] = target_persons.opm_pov_thresh - target_persons.ed_debt_income_10
  ed_debt_gap_billions_10 = (((target_persons.ed_debt_poor_10 * target_persons.ed_debt_poverty_gap_10 * target_persons.wgt_numper).sum()) / 1e9).round(1)

  # Convert and round
  ed_debt_change_10 = ed_debt_change_10.astype(int)
  ubi_change_10 = ubi_change_10.astype(int)

  return pd.Series([ed_debt_change_10, ubi_change_10, percent_better_off_with_ed_10, 
                    percent_initial_debt_10, percent_ubi_debt_10, 
                    percent_no_ed_debt_debt_10, percent_ubi_poor_10, percent_ed_debt_poor_10,
                    ubi_gap_billions_10, ed_debt_gap_billions_10])

In [76]:
def age_row_10(row):
  return age_fun_10(row.agecl)

In [77]:
age[['ed_debt_change_10', 'ubi_change_10', 'percent_better_off_with_ed_10',
        'in_debt_10', 'in_debt_ubi_10', 'in_debt_no_ed_debt_10',
      'percent_ubi_poor_10', 'percent_ed_debt_poor_10', 
     'ubi_gap_billions_10', 'ed_debt_gap_billions_10']] = age.apply(age_row_10, axis=1)

In [78]:
def income_fun_10(income_pp_quintile):
  target_persons = scf[scf.income_pp_quintile==income_pp_quintile].copy(deep=True)

  # Calculate average benefit by reform
  start = mdf.weighted_mean(target_persons, 'networth_pp', 'wgt_numper')
  ed_debt = mdf.weighted_mean(target_persons, 'no_debt_networth_pp_10', 'wgt_numper')
  ubi = mdf.weighted_mean(target_persons, 'ubi_networth_pp_10', 'wgt_numper')

  ed_debt_change_10 = ed_debt - start
  ubi_change_10 = ubi - start

  # Calculate percent better off with ed debt cancellation
  target_persons['better_with_ed_10'] = target_persons.ubi_networth_pp_10 < target_persons.no_debt_networth_pp_10
  total_better_off_with_ed_10 = (target_persons.better_with_ed_10 * target_persons.wgt_numper).sum()
  target_pop = target_persons.wgt_numper.sum()
  percent_better_off_with_ed_10 = (total_better_off_with_ed_10 / target_pop * 100).round(1)

  # Calculate percent out of debt
  target_persons['initial_debt_10'] = target_persons.networth < 0
  target_persons['ubi_debt_10'] = target_persons.ubi_networth_pp < 0
  target_persons['no_ed_debt_debt_10'] = target_persons.no_debt_networth_pp < 0

  percent_initial_debt_10 = (((target_persons.initial_debt_10 
                           * target_persons.wgt_numper).sum()) / target_pop 
                          * 100).round(1)
  percent_ubi_debt_10 = (((target_persons.ubi_debt_10 
                           * target_persons.wgt_numper).sum()) / target_pop
                      * 100).round(1)
  percent_no_ed_debt_debt_10 = (((target_persons.no_ed_debt_debt_10 
                           * target_persons.wgt_numper).sum()) / target_pop
                           * 100).round(1)

  # Calculate percent poor
  total_original_poor_10 = (target_persons.original_poor * target_persons.wgt_numper).sum()
  percent_original_poor_10 = (total_original_poor_10 / target_pop * 100).round(1)

  target_persons['ubi_income_10'] = target_persons.total_ubi_10 + target_persons.income
  target_persons['ubi_poor_10'] = target_persons.ubi_income_10 < target_persons.opm_pov_thresh
  ubi_total_poor_10 = (target_persons.wgt_numper * target_persons.ubi_poor_10).sum()
  ubi_percent_poor_10 = ubi_total_poor_10 / target_pop
  percent_ubi_poor_10 = (ubi_percent_poor_10 * 100).round(1)

  target_persons['ed_debt_income_10'] = target_persons.difference_10 + target_persons.income
  target_persons['ed_debt_poor_10'] = target_persons.ed_debt_income_10 < target_persons.opm_pov_thresh
  ed_debt_total_poor_10 = (target_persons.wgt_numper * target_persons.ed_debt_poor_10).sum()
  ed_debt_percent_poor_10 = ed_debt_total_poor_10 / target_pop
  percent_ed_debt_poor_10 = (ed_debt_percent_poor_10 * 100).round(1)

  # Calculate poverty gap
  target_persons['ubi_poverty_gap_10'] = target_persons.opm_pov_thresh - target_persons.ubi_income_10
  ubi_gap_billions_10 = (((target_persons.ubi_poor_10 * target_persons.ubi_poverty_gap_10 * target_persons.wgt_numper).sum()) / 1e9).round(1)

  target_persons['ed_debt_poverty_gap_10'] = target_persons.opm_pov_thresh - target_persons.ed_debt_income_10
  ed_debt_gap_billions_10 = (((target_persons.ed_debt_poor_10 * target_persons.ed_debt_poverty_gap_10 * target_persons.wgt_numper).sum()) / 1e9).round(1)

  # Convert and round
  ed_debt_change_10 = ed_debt_change_10.astype(int)
  ubi_change_10 = ubi_change_10.astype(int)

  return pd.Series([ed_debt_change_10, ubi_change_10, percent_better_off_with_ed_10, 
                    percent_initial_debt_10, percent_ubi_debt_10, 
                    percent_no_ed_debt_debt_10, percent_ubi_poor_10, percent_ed_debt_poor_10,
                    ubi_gap_billions_10, ed_debt_gap_billions_10])

In [79]:
def income_row_10(row):
  return income_fun_10(row.income_pp_quintile)

In [80]:
income[['ed_debt_change_10', 'ubi_change_10', 'percent_better_off_with_ed_10',
        'in_debt_10', 'in_debt_ubi_10', 'in_debt_no_ed_debt_10',
      'percent_ubi_poor_10', 'percent_ed_debt_poor_10', 
     'ubi_gap_billions_10', 'ed_debt_gap_billions_10']] = income.apply(income_row_10, axis=1)

In [81]:
def networth_fun_10(networth_pp_quintile2):
  target_persons = scf[scf.networth_pp_quintile2==networth_pp_quintile2].copy(deep=True)

  # Calculate average benefit by reform
  start = mdf.weighted_mean(target_persons, 'networth_pp', 'wgt_numper')
  ed_debt = mdf.weighted_mean(target_persons, 'no_debt_networth_pp_10', 'wgt_numper')
  ubi = mdf.weighted_mean(target_persons, 'ubi_networth_pp_10', 'wgt_numper')

  ed_debt_change_10 = ed_debt - start
  ubi_change_10 = ubi - start

  # Calculate percent better off with ed debt cancellation
  target_persons['better_with_ed_10'] = target_persons.ubi_networth_pp_10 < target_persons.no_debt_networth_pp_10
  total_better_off_with_ed_10 = (target_persons.better_with_ed_10 * target_persons.wgt_numper).sum()
  target_pop = target_persons.wgt_numper.sum()
  percent_better_off_with_ed_10 = (total_better_off_with_ed_10 / target_pop * 100).round(1)

  # Calculate percent out of debt
  target_persons['initial_debt_10'] = target_persons.networth < 0
  target_persons['ubi_debt_10'] = target_persons.ubi_networth_pp < 0
  target_persons['no_ed_debt_debt_10'] = target_persons.no_debt_networth_pp < 0

  percent_initial_debt_10 = (((target_persons.initial_debt_10 
                           * target_persons.wgt_numper).sum()) / target_pop 
                          * 100).round(1)
  percent_ubi_debt_10 = (((target_persons.ubi_debt_10 
                           * target_persons.wgt_numper).sum()) / target_pop
                      * 100).round(1)
  percent_no_ed_debt_debt_10 = (((target_persons.no_ed_debt_debt_10 
                           * target_persons.wgt_numper).sum()) / target_pop
                           * 100).round(1)

  # Calculate percent poor
  total_original_poor_10 = (target_persons.original_poor * target_persons.wgt_numper).sum()
  percent_original_poor_10 = (total_original_poor_10 / target_pop * 100).round(1)

  target_persons['ubi_income_10'] = target_persons.total_ubi_10 + target_persons.income
  target_persons['ubi_poor_10'] = target_persons.ubi_income_10 < target_persons.opm_pov_thresh
  ubi_total_poor_10 = (target_persons.wgt_numper * target_persons.ubi_poor_10).sum()
  ubi_percent_poor_10 = ubi_total_poor_10 / target_pop
  percent_ubi_poor_10 = (ubi_percent_poor_10 * 100).round(1)

  target_persons['ed_debt_income_10'] = target_persons.difference_10 + target_persons.income
  target_persons['ed_debt_poor_10'] = target_persons.ed_debt_income_10 < target_persons.opm_pov_thresh
  ed_debt_total_poor_10 = (target_persons.wgt_numper * target_persons.ed_debt_poor_10).sum()
  ed_debt_percent_poor_10 = ed_debt_total_poor_10 / target_pop
  percent_ed_debt_poor_10 = (ed_debt_percent_poor_10 * 100).round(1)

  # Calculate poverty gap
  target_persons['ubi_poverty_gap_10'] = target_persons.opm_pov_thresh - target_persons.ubi_income_10
  ubi_gap_billions_10 = (((target_persons.ubi_poor_10 * target_persons.ubi_poverty_gap_10 * target_persons.wgt_numper).sum()) / 1e9).round(1)

  target_persons['ed_debt_poverty_gap_10'] = target_persons.opm_pov_thresh - target_persons.ed_debt_income_10
  ed_debt_gap_billions_10 = (((target_persons.ed_debt_poor_10 * target_persons.ed_debt_poverty_gap_10 * target_persons.wgt_numper).sum()) / 1e9).round(1)

  # Convert and round
  ed_debt_change_10 = ed_debt_change_10.astype(int)
  ubi_change_10 = ubi_change_10.astype(int)

  return pd.Series([ed_debt_change_10, ubi_change_10, percent_better_off_with_ed_10, 
                    percent_initial_debt_10, percent_ubi_debt_10, 
                    percent_no_ed_debt_debt_10, percent_ubi_poor_10, percent_ed_debt_poor_10,
                    ubi_gap_billions_10, ed_debt_gap_billions_10])

In [82]:
def networth_row_10(row):
  return networth_fun_10(row.networth_pp_quintile2)

In [83]:
networth[['ed_debt_change_10', 'ubi_change_10', 'percent_better_off_with_ed_10',
        'in_debt_10', 'in_debt_ubi_10', 'in_debt_no_ed_debt_10',
      'percent_ubi_poor_10', 'percent_ed_debt_poor_10', 
     'ubi_gap_billions_10', 'ed_debt_gap_billions_10']] = networth.apply(networth_row_10, axis=1)

In [84]:
def poor_fun_10(original_poor):
  target_persons = scf[scf.original_poor==original_poor].copy(deep=True)

  # Calculate average benefit by reform
  start = mdf.weighted_mean(target_persons, 'networth_pp', 'wgt_numper')
  ed_debt = mdf.weighted_mean(target_persons, 'no_debt_networth_pp_10', 'wgt_numper')
  ubi = mdf.weighted_mean(target_persons, 'ubi_networth_pp_10', 'wgt_numper')

  ed_debt_change_10 = ed_debt - start
  ubi_change_10 = ubi - start

  # Calculate percent better off with ed debt cancellation
  target_persons['better_with_ed_10'] = target_persons.ubi_networth_pp_10 < target_persons.no_debt_networth_pp_10
  total_better_off_with_ed_10 = (target_persons.better_with_ed_10 * target_persons.wgt_numper).sum()
  target_pop = target_persons.wgt_numper.sum()
  percent_better_off_with_ed_10 = (total_better_off_with_ed_10 / target_pop * 100).round(1)

  # Calculate percent out of debt
  target_persons['initial_debt_10'] = target_persons.networth < 0
  target_persons['ubi_debt_10'] = target_persons.ubi_networth_pp < 0
  target_persons['no_ed_debt_debt_10'] = target_persons.no_debt_networth_pp < 0

  percent_initial_debt_10 = (((target_persons.initial_debt_10 
                           * target_persons.wgt_numper).sum()) / target_pop 
                          * 100).round(1)
  percent_ubi_debt_10 = (((target_persons.ubi_debt_10 
                           * target_persons.wgt_numper).sum()) / target_pop
                      * 100).round(1)
  percent_no_ed_debt_debt_10 = (((target_persons.no_ed_debt_debt_10 
                           * target_persons.wgt_numper).sum()) / target_pop
                           * 100).round(1)

  # Calculate percent poor
  total_original_poor_10 = (target_persons.original_poor * target_persons.wgt_numper).sum()
  percent_original_poor_10 = (total_original_poor_10 / target_pop * 100).round(1)

  target_persons['ubi_income_10'] = target_persons.total_ubi_10 + target_persons.income
  target_persons['ubi_poor_10'] = target_persons.ubi_income_10 < target_persons.opm_pov_thresh
  ubi_total_poor_10 = (target_persons.wgt_numper * target_persons.ubi_poor_10).sum()
  ubi_percent_poor_10 = ubi_total_poor_10 / target_pop
  percent_ubi_poor_10 = (ubi_percent_poor_10 * 100).round(1)

  target_persons['ed_debt_income_10'] = target_persons.difference_10 + target_persons.income
  target_persons['ed_debt_poor_10'] = target_persons.ed_debt_income_10 < target_persons.opm_pov_thresh
  ed_debt_total_poor_10 = (target_persons.wgt_numper * target_persons.ed_debt_poor_10).sum()
  ed_debt_percent_poor_10 = ed_debt_total_poor_10 / target_pop
  percent_ed_debt_poor_10 = (ed_debt_percent_poor_10 * 100).round(1)

  # Calculate poverty gap
  target_persons['ubi_poverty_gap_10'] = target_persons.opm_pov_thresh - target_persons.ubi_income_10
  ubi_gap_billions_10 = (((target_persons.ubi_poor_10 * target_persons.ubi_poverty_gap_10 * target_persons.wgt_numper).sum()) / 1e9).round(1)

  target_persons['ed_debt_poverty_gap_10'] = target_persons.opm_pov_thresh - target_persons.ed_debt_income_10
  ed_debt_gap_billions_10 = (((target_persons.ed_debt_poor_10 * target_persons.ed_debt_poverty_gap_10 * target_persons.wgt_numper).sum()) / 1e9).round(1)

  # Convert and round
  ed_debt_change_10 = ed_debt_change_10.astype(int)
  ubi_change_10 = ubi_change_10.astype(int)

  return pd.Series([ed_debt_change_10, ubi_change_10, percent_better_off_with_ed_10, 
                    percent_initial_debt_10, percent_ubi_debt_10, 
                    percent_no_ed_debt_debt_10, percent_ubi_poor_10, percent_ed_debt_poor_10,
                    ubi_gap_billions_10, ed_debt_gap_billions_10])

In [85]:
def poor_row_10(row):
  return poor_fun_10(row.original_poor)

In [86]:
poor[['ed_debt_change_10', 'ubi_change_10', 'percent_better_off_with_ed_10',
        'in_debt_10', 'in_debt_ubi_10', 'in_debt_no_ed_debt_10',
      'percent_ubi_poor_10', 'percent_ed_debt_poor_10', 
     'ubi_gap_billions_10', 'ed_debt_gap_billions_10']] = poor.apply(poor_row_10, axis=1)

## Groupings

In [87]:
def all_fun(index_num):
  target_persons = scf.copy(deep=True)

  # Calculate average benefit by reform
  start = mdf.weighted_mean(target_persons, 'networth_pp', 'wgt_numper')
  ed_debt = mdf.weighted_mean(target_persons, 'no_debt_networth_pp', 'wgt_numper')
  ubi = mdf.weighted_mean(target_persons, 'ubi_networth_pp', 'wgt_numper')

  ed_debt_change = ed_debt - start
  ubi_change = ubi - start

  # Calculate percent better off with ed debt cancellation
  target_persons['better_with_ed'] = target_persons.ubi_networth_pp < target_persons.no_debt_networth_pp
  total_better_off_with_ed = (target_persons.better_with_ed * target_persons.wgt_numper).sum()
  target_pop = target_persons.wgt_numper.sum()
  percent_better_off_with_ed = (total_better_off_with_ed / target_pop * 100).round(1)

  # Calculate percent out of debt
  target_persons['initial_debt'] = target_persons.networth_pp < 0
  target_persons['ubi_debt'] = target_persons.ubi_networth_pp < 0
  target_persons['no_ed_debt_debt'] = target_persons.no_debt_networth_pp < 0

  percent_initial_debt = (((target_persons.initial_debt 
                           * target_persons.wgt_numper).sum()) / target_pop 
                          * 100).round(1)
  percent_ubi_debt = (((target_persons.ubi_debt 
                           * target_persons.wgt_numper).sum()) / target_pop
                      * 100).round(1)
  percent_no_ed_debt_debt = (((target_persons.no_ed_debt_debt 
                           * target_persons.wgt_numper).sum()) / target_pop
                           * 100).round(1)

  # Calculate percent poor
  total_original_poor = (target_persons.original_poor * target_persons.wgt_numper).sum()
  percent_original_poor = (total_original_poor / target_pop * 100).round(1)

  target_persons['ubi_income'] = target_persons.total_ubi + target_persons.income
  target_persons['ubi_poor'] = target_persons.ubi_income < target_persons.opm_pov_thresh
  ubi_total_poor = (target_persons.wgt_numper * target_persons.ubi_poor).sum()
  ubi_percent_poor = ubi_total_poor / target_pop
  percent_ubi_poor = (ubi_percent_poor * 100).round(1)

  target_persons['ed_debt_income'] = target_persons.edn_inst + target_persons.income
  target_persons['ed_debt_poor'] = target_persons.ed_debt_income < target_persons.opm_pov_thresh
  ed_debt_total_poor = (target_persons.wgt_numper * target_persons.ed_debt_poor).sum()
  ed_debt_percent_poor = ed_debt_total_poor / target_pop
  percent_ed_debt_poor = (ed_debt_percent_poor * 100).round(1)

  # Calculate poverty gap
  target_persons['original_poverty_gap'] = target_persons.opm_pov_thresh - target_persons.income
  original_gap_billions = (((target_persons.original_poor * target_persons.original_poverty_gap * target_persons.wgt_numper).sum()) / 1e9).round(1)

  target_persons['ubi_poverty_gap'] = target_persons.opm_pov_thresh - target_persons.ubi_income
  ubi_gap_billions = (((target_persons.ubi_poor * target_persons.ubi_poverty_gap * target_persons.wgt_numper).sum()) / 1e9).round(1)

  target_persons['ed_debt_poverty_gap'] = target_persons.opm_pov_thresh - target_persons.ed_debt_income
  ed_debt_gap_billions = (((target_persons.ed_debt_poor * target_persons.ed_debt_poverty_gap * target_persons.wgt_numper).sum()) / 1e9).round(1)

  # Convert and round
  ed_debt_change = ed_debt_change.astype(int)
  ubi_change = ubi_change.astype(int)

  return pd.Series([ed_debt_change, ubi_change, percent_better_off_with_ed, 
                    percent_initial_debt, percent_ubi_debt, 
                    percent_no_ed_debt_debt, percent_original_poor, 
                    percent_ubi_poor, percent_ed_debt_poor,
                    original_gap_billions, ubi_gap_billions, ed_debt_gap_billions])

In [88]:
def all_row(row):
  return all_fun(row.index)

In [89]:
all[['ed_debt_change', 'ubi_change', 'percent_better_off_with_ed',
        'in_debt', 'in_debt_ubi', 'in_debt_no_ed_debt',
        'percent_original_poor', 'percent_ubi_poor', 'percent_ed_debt_poor', 
     'original_gap_billions', 'ubi_gap_billions', 'ed_debt_gap_billions']] = all.apply(all_row, axis=1)

In [90]:
def race_fun(race):
  target_persons = scf[scf.race==race].copy(deep=True)

  # Calculate average benefit by reform
  start = mdf.weighted_mean(target_persons, 'networth_pp', 'wgt_numper')
  ed_debt = mdf.weighted_mean(target_persons, 'no_debt_networth_pp', 'wgt_numper')
  ubi = mdf.weighted_mean(target_persons, 'ubi_networth_pp', 'wgt_numper')

  ed_debt_change = ed_debt - start
  ubi_change = ubi - start

  # Calculate percent better off with ed debt cancellation
  target_persons['better_with_ed'] = target_persons.ubi_networth_pp < target_persons.no_debt_networth_pp
  total_better_off_with_ed = (target_persons.better_with_ed * target_persons.wgt_numper).sum()
  target_pop = target_persons.wgt_numper.sum()
  percent_better_off_with_ed = (total_better_off_with_ed / target_pop * 100).round(1)

  # Calculate percent out of debt
  target_persons['initial_debt'] = target_persons.networth_pp < 0
  target_persons['ubi_debt'] = target_persons.ubi_networth_pp < 0
  target_persons['no_ed_debt_debt'] = target_persons.no_debt_networth_pp < 0

  percent_initial_debt = (((target_persons.initial_debt 
                           * target_persons.wgt_numper).sum()) / target_pop 
                          * 100).round(1)
  percent_ubi_debt = (((target_persons.ubi_debt 
                           * target_persons.wgt_numper).sum()) / target_pop
                      * 100).round(1)
  percent_no_ed_debt_debt = (((target_persons.no_ed_debt_debt 
                           * target_persons.wgt_numper).sum()) / target_pop
                           * 100).round(1)

  # Calculate percent poor
  total_original_poor = (target_persons.original_poor * target_persons.wgt_numper).sum()
  percent_original_poor = (total_original_poor / target_pop * 100).round(1)

  target_persons['ubi_income'] = target_persons.total_ubi + target_persons.income
  target_persons['ubi_poor'] = target_persons.ubi_income < target_persons.opm_pov_thresh
  ubi_total_poor = (target_persons.wgt_numper * target_persons.ubi_poor).sum()
  ubi_percent_poor = ubi_total_poor / target_pop
  percent_ubi_poor = (ubi_percent_poor * 100).round(1)

  target_persons['ed_debt_income'] = target_persons.edn_inst + target_persons.income
  target_persons['ed_debt_poor'] = target_persons.ed_debt_income < target_persons.opm_pov_thresh
  ed_debt_total_poor = (target_persons.wgt_numper * target_persons.ed_debt_poor).sum()
  ed_debt_percent_poor = ed_debt_total_poor / target_pop
  percent_ed_debt_poor = (ed_debt_percent_poor * 100).round(1)

  # Calculate poverty gap
  target_persons['original_poverty_gap'] = target_persons.opm_pov_thresh - target_persons.income
  original_gap_billions = (((target_persons.original_poor * target_persons.original_poverty_gap * target_persons.wgt_numper).sum()) / 1e9).round(1)

  target_persons['ubi_poverty_gap'] = target_persons.opm_pov_thresh - target_persons.ubi_income
  ubi_gap_billions = (((target_persons.ubi_poor * target_persons.ubi_poverty_gap * target_persons.wgt_numper).sum()) / 1e9).round(1)

  target_persons['ed_debt_poverty_gap'] = target_persons.opm_pov_thresh - target_persons.ed_debt_income
  ed_debt_gap_billions = (((target_persons.ed_debt_poor * target_persons.ed_debt_poverty_gap * target_persons.wgt_numper).sum()) / 1e9).round(1)

  # Convert and round
  ed_debt_change = ed_debt_change.astype(int)
  ubi_change = ubi_change.astype(int)

  return pd.Series([ed_debt_change, ubi_change, percent_better_off_with_ed, 
                    percent_initial_debt, percent_ubi_debt, 
                    percent_no_ed_debt_debt, percent_original_poor, 
                    percent_ubi_poor, percent_ed_debt_poor,
                    original_gap_billions, ubi_gap_billions, ed_debt_gap_billions])

In [91]:
def race_row(row):
  return race_fun(row.race)

In [92]:
race[['ed_debt_change', 'ubi_change', 'percent_better_off_with_ed',
        'in_debt', 'in_debt_ubi', 'in_debt_no_ed_debt',
        'percent_original_poor', 'percent_ubi_poor', 'percent_ed_debt_poor', 
     'original_gap_billions', 'ubi_gap_billions', 'ed_debt_gap_billions']] = race.apply(race_row, axis=1)

In [93]:
def education_fun(edcl):
  target_persons = scf[scf.edcl==edcl].copy(deep=True)

  # Calculate average benefit by reform
  start = mdf.weighted_mean(target_persons, 'networth_pp', 'wgt_numper')
  ed_debt = mdf.weighted_mean(target_persons, 'no_debt_networth_pp', 'wgt_numper')
  ubi = mdf.weighted_mean(target_persons, 'ubi_networth_pp', 'wgt_numper')

  ed_debt_change = ed_debt - start
  ubi_change = ubi - start

  # Calculate percent better off with ed debt cancellation
  target_persons['better_with_ed'] = target_persons.ubi_networth_pp < target_persons.no_debt_networth_pp
  total_better_off_with_ed = (target_persons.better_with_ed * target_persons.wgt_numper).sum()
  target_pop = target_persons.wgt_numper.sum()
  percent_better_off_with_ed = (total_better_off_with_ed / target_pop * 100).round(1)

  # Calculate percent out of debt
  target_persons['initial_debt'] = target_persons.networth_pp < 0
  target_persons['ubi_debt'] = target_persons.ubi_networth_pp < 0
  target_persons['no_ed_debt_debt'] = target_persons.no_debt_networth_pp < 0

  percent_initial_debt = (((target_persons.initial_debt 
                           * target_persons.wgt_numper).sum()) / target_pop 
                          * 100).round(1)
  percent_ubi_debt = (((target_persons.ubi_debt 
                           * target_persons.wgt_numper).sum()) / target_pop
                      * 100).round(1)
  percent_no_ed_debt_debt = (((target_persons.no_ed_debt_debt 
                           * target_persons.wgt_numper).sum()) / target_pop
                           * 100).round(1)

  # Calculate percent poor
  total_original_poor = (target_persons.original_poor * target_persons.wgt_numper).sum()
  percent_original_poor = (total_original_poor / target_pop * 100).round(1)

  target_persons['ubi_income'] = target_persons.total_ubi + target_persons.income
  target_persons['ubi_poor'] = target_persons.ubi_income < target_persons.opm_pov_thresh
  ubi_total_poor = (target_persons.wgt_numper * target_persons.ubi_poor).sum()
  ubi_percent_poor = ubi_total_poor / target_pop
  percent_ubi_poor = (ubi_percent_poor * 100).round(1)

  target_persons['ed_debt_income'] = target_persons.edn_inst + target_persons.income
  target_persons['ed_debt_poor'] = target_persons.ed_debt_income < target_persons.opm_pov_thresh
  ed_debt_total_poor = (target_persons.wgt_numper * target_persons.ed_debt_poor).sum()
  ed_debt_percent_poor = ed_debt_total_poor / target_pop
  percent_ed_debt_poor = (ed_debt_percent_poor * 100).round(1)

  # Calculate poverty gap
  target_persons['original_poverty_gap'] = target_persons.opm_pov_thresh - target_persons.income
  original_gap_billions = (((target_persons.original_poor * target_persons.original_poverty_gap * target_persons.wgt_numper).sum()) / 1e9).round(1)

  target_persons['ubi_poverty_gap'] = target_persons.opm_pov_thresh - target_persons.ubi_income
  ubi_gap_billions = (((target_persons.ubi_poor * target_persons.ubi_poverty_gap * target_persons.wgt_numper).sum()) / 1e9).round(1)

  target_persons['ed_debt_poverty_gap'] = target_persons.opm_pov_thresh - target_persons.ed_debt_income
  ed_debt_gap_billions = (((target_persons.ed_debt_poor * target_persons.ed_debt_poverty_gap * target_persons.wgt_numper).sum()) / 1e9).round(1)

  # Convert and round
  ed_debt_change = ed_debt_change.astype(int)
  ubi_change = ubi_change.astype(int)

  return pd.Series([ed_debt_change, ubi_change, percent_better_off_with_ed, 
                    percent_initial_debt, percent_ubi_debt, 
                    percent_no_ed_debt_debt, percent_original_poor, 
                    percent_ubi_poor, percent_ed_debt_poor,
                    original_gap_billions, ubi_gap_billions, ed_debt_gap_billions])

In [94]:
def education_row(row):
  return education_fun(row.edcl)

In [95]:
education[['ed_debt_change', 'ubi_change', 'percent_better_off_with_ed',
        'in_debt', 'in_debt_ubi', 'in_debt_no_ed_debt',
        'percent_original_poor', 'percent_ubi_poor', 'percent_ed_debt_poor', 
     'original_gap_billions', 'ubi_gap_billions', 'ed_debt_gap_billions']] = education.apply(education_row, axis=1)

In [96]:
def income_fun(income_pp_quintile):
  target_persons = scf[scf.income_pp_quintile==income_pp_quintile].copy(deep=True)

  # Calculate average benefit by reform
  start = mdf.weighted_mean(target_persons, 'networth_pp', 'wgt_numper')
  ed_debt = mdf.weighted_mean(target_persons, 'no_debt_networth_pp', 'wgt_numper')
  ubi = mdf.weighted_mean(target_persons, 'ubi_networth_pp', 'wgt_numper')

  ed_debt_change = ed_debt - start
  ubi_change = ubi - start

  # Calculate percent better off with ed debt cancellation
  target_persons['better_with_ed'] = target_persons.ubi_networth_pp < target_persons.no_debt_networth_pp
  total_better_off_with_ed = (target_persons.better_with_ed * target_persons.wgt_numper).sum()
  target_pop = target_persons.wgt_numper.sum()
  percent_better_off_with_ed = (total_better_off_with_ed / target_pop * 100).round(1)

  # Calculate percent out of debt
  target_persons['initial_debt'] = target_persons.networth_pp < 0
  target_persons['ubi_debt'] = target_persons.ubi_networth_pp < 0
  target_persons['no_ed_debt_debt'] = target_persons.no_debt_networth_pp < 0

  percent_initial_debt = (((target_persons.initial_debt 
                           * target_persons.wgt_numper).sum()) / target_pop 
                          * 100).round(1)
  percent_ubi_debt = (((target_persons.ubi_debt 
                           * target_persons.wgt_numper).sum()) / target_pop
                      * 100).round(1)
  percent_no_ed_debt_debt = (((target_persons.no_ed_debt_debt 
                           * target_persons.wgt_numper).sum()) / target_pop
                           * 100).round(1)

  # Calculate percent poor
  total_original_poor = (target_persons.original_poor * target_persons.wgt_numper).sum()
  percent_original_poor = (total_original_poor / target_pop * 100).round(1)

  target_persons['ubi_income'] = target_persons.total_ubi + target_persons.income
  target_persons['ubi_poor'] = target_persons.ubi_income < target_persons.opm_pov_thresh
  ubi_total_poor = (target_persons.wgt_numper * target_persons.ubi_poor).sum()
  ubi_percent_poor = ubi_total_poor / target_pop
  percent_ubi_poor = (ubi_percent_poor * 100).round(1)

  target_persons['ed_debt_income'] = target_persons.edn_inst + target_persons.income
  target_persons['ed_debt_poor'] = target_persons.ed_debt_income < target_persons.opm_pov_thresh
  ed_debt_total_poor = (target_persons.wgt_numper * target_persons.ed_debt_poor).sum()
  ed_debt_percent_poor = ed_debt_total_poor / target_pop
  percent_ed_debt_poor = (ed_debt_percent_poor * 100).round(1)

  # Calculate poverty gap
  target_persons['original_poverty_gap'] = target_persons.opm_pov_thresh - target_persons.income
  original_gap_billions = (((target_persons.original_poor * target_persons.original_poverty_gap * target_persons.wgt_numper).sum()) / 1e9).round(1)

  target_persons['ubi_poverty_gap'] = target_persons.opm_pov_thresh - target_persons.ubi_income
  ubi_gap_billions = (((target_persons.ubi_poor * target_persons.ubi_poverty_gap * target_persons.wgt_numper).sum()) / 1e9).round(1)

  target_persons['ed_debt_poverty_gap'] = target_persons.opm_pov_thresh - target_persons.ed_debt_income
  ed_debt_gap_billions = (((target_persons.ed_debt_poor * target_persons.ed_debt_poverty_gap * target_persons.wgt_numper).sum()) / 1e9).round(1)

  # Convert and round
  ed_debt_change = ed_debt_change.astype(int)
  ubi_change = ubi_change.astype(int)

  return pd.Series([ed_debt_change, ubi_change, percent_better_off_with_ed, 
                    percent_initial_debt, percent_ubi_debt, 
                    percent_no_ed_debt_debt, percent_original_poor, 
                    percent_ubi_poor, percent_ed_debt_poor,
                    original_gap_billions, ubi_gap_billions, ed_debt_gap_billions])

In [97]:
def income_row(row):
  return income_fun(row.income_pp_quintile)

In [98]:
income[['ed_debt_change', 'ubi_change', 'percent_better_off_with_ed',
        'in_debt', 'in_debt_ubi', 'in_debt_no_ed_debt',
        'percent_original_poor', 'percent_ubi_poor', 'percent_ed_debt_poor', 
     'original_gap_billions', 'ubi_gap_billions', 'ed_debt_gap_billions']] = income.apply(income_row, axis=1)

In [99]:
def age_fun(agecl):
  target_persons = scf[scf.agecl==agecl].copy(deep=True)

  # Calculate average benefit by reform
  start = mdf.weighted_mean(target_persons, 'networth_pp', 'wgt_numper')
  ed_debt = mdf.weighted_mean(target_persons, 'no_debt_networth_pp', 'wgt_numper')
  ubi = mdf.weighted_mean(target_persons, 'ubi_networth_pp', 'wgt_numper')

  ed_debt_change = ed_debt - start
  ubi_change = ubi - start

  # Calculate percent better off with ed debt cancellation
  target_persons['better_with_ed'] = target_persons.ubi_networth_pp < target_persons.no_debt_networth_pp
  total_better_off_with_ed = (target_persons.better_with_ed * target_persons.wgt_numper).sum()
  target_pop = target_persons.wgt_numper.sum()
  percent_better_off_with_ed = (total_better_off_with_ed / target_pop * 100).round(1)

  # Calculate percent out of debt
  target_persons['initial_debt'] = target_persons.networth_pp < 0
  target_persons['ubi_debt'] = target_persons.ubi_networth_pp < 0
  target_persons['no_ed_debt_debt'] = target_persons.no_debt_networth_pp < 0

  percent_initial_debt = (((target_persons.initial_debt 
                           * target_persons.wgt_numper).sum()) / target_pop 
                          * 100).round(1)
  percent_ubi_debt = (((target_persons.ubi_debt 
                           * target_persons.wgt_numper).sum()) / target_pop
                      * 100).round(1)
  percent_no_ed_debt_debt = (((target_persons.no_ed_debt_debt 
                           * target_persons.wgt_numper).sum()) / target_pop
                           * 100).round(1)

  # Calculate percent poor
  total_original_poor = (target_persons.original_poor * target_persons.wgt_numper).sum()
  percent_original_poor = (total_original_poor / target_pop * 100).round(1)

  target_persons['ubi_income'] = target_persons.total_ubi + target_persons.income
  target_persons['ubi_poor'] = target_persons.ubi_income < target_persons.opm_pov_thresh
  ubi_total_poor = (target_persons.wgt_numper * target_persons.ubi_poor).sum()
  ubi_percent_poor = ubi_total_poor / target_pop
  percent_ubi_poor = (ubi_percent_poor * 100).round(1)

  target_persons['ed_debt_income'] = target_persons.edn_inst + target_persons.income
  target_persons['ed_debt_poor'] = target_persons.ed_debt_income < target_persons.opm_pov_thresh
  ed_debt_total_poor = (target_persons.wgt_numper * target_persons.ed_debt_poor).sum()
  ed_debt_percent_poor = ed_debt_total_poor / target_pop
  percent_ed_debt_poor = (ed_debt_percent_poor * 100).round(1)

  # Calculate poverty gap
  target_persons['original_poverty_gap'] = target_persons.opm_pov_thresh - target_persons.income
  original_gap_billions = (((target_persons.original_poor * target_persons.original_poverty_gap * target_persons.wgt_numper).sum()) / 1e9).round(1)

  target_persons['ubi_poverty_gap'] = target_persons.opm_pov_thresh - target_persons.ubi_income
  ubi_gap_billions = (((target_persons.ubi_poor * target_persons.ubi_poverty_gap * target_persons.wgt_numper).sum()) / 1e9).round(1)

  target_persons['ed_debt_poverty_gap'] = target_persons.opm_pov_thresh - target_persons.ed_debt_income
  ed_debt_gap_billions = (((target_persons.ed_debt_poor * target_persons.ed_debt_poverty_gap * target_persons.wgt_numper).sum()) / 1e9).round(1)

  # Convert and round
  ed_debt_change = ed_debt_change.astype(int)
  ubi_change = ubi_change.astype(int)

  return pd.Series([ed_debt_change, ubi_change, percent_better_off_with_ed, 
                    percent_initial_debt, percent_ubi_debt, 
                    percent_no_ed_debt_debt, percent_original_poor, 
                    percent_ubi_poor, percent_ed_debt_poor,
                    original_gap_billions, ubi_gap_billions, ed_debt_gap_billions])

In [100]:
def age_row(row):
  return age_fun(row.agecl)

In [101]:
age[['ed_debt_change', 'ubi_change', 'percent_better_off_with_ed',
        'in_debt', 'in_debt_ubi', 'in_debt_no_ed_debt',
        'percent_original_poor', 'percent_ubi_poor', 'percent_ed_debt_poor', 
     'original_gap_billions', 'ubi_gap_billions', 'ed_debt_gap_billions']] = age.apply(age_row, axis=1)

In [102]:
def networth_fun(networth_pp_quintile2):
  target_persons = scf[scf.networth_pp_quintile2==networth_pp_quintile2].copy(deep=True)

  # Calculate average benefit by reform
  start = mdf.weighted_mean(target_persons, 'networth_pp', 'wgt_numper')
  ed_debt = mdf.weighted_mean(target_persons, 'no_debt_networth_pp', 'wgt_numper')
  ubi = mdf.weighted_mean(target_persons, 'ubi_networth_pp', 'wgt_numper')

  ed_debt_change = ed_debt - start
  ubi_change = ubi - start

  # Calculate percent better off with ed debt cancellation
  target_persons['better_with_ed'] = target_persons.ubi_networth_pp < target_persons.no_debt_networth_pp
  total_better_off_with_ed = (target_persons.better_with_ed * target_persons.wgt_numper).sum()
  target_pop = target_persons.wgt_numper.sum()
  percent_better_off_with_ed = (total_better_off_with_ed / target_pop * 100).round(1)

  # Calculate percent out of debt
  target_persons['initial_debt'] = target_persons.networth_pp < 0
  target_persons['ubi_debt'] = target_persons.ubi_networth_pp < 0
  target_persons['no_ed_debt_debt'] = target_persons.no_debt_networth_pp < 0

  percent_initial_debt = (((target_persons.initial_debt 
                           * target_persons.wgt_numper).sum()) / target_pop 
                          * 100).round(1)
  percent_ubi_debt = (((target_persons.ubi_debt 
                           * target_persons.wgt_numper).sum()) / target_pop
                      * 100).round(1)
  percent_no_ed_debt_debt = (((target_persons.no_ed_debt_debt 
                           * target_persons.wgt_numper).sum()) / target_pop
                           * 100).round(1)

  # Calculate percent poor
  total_original_poor = (target_persons.original_poor * target_persons.wgt_numper).sum()
  percent_original_poor = (total_original_poor / target_pop * 100).round(1)

  target_persons['ubi_income'] = target_persons.total_ubi + target_persons.income
  target_persons['ubi_poor'] = target_persons.ubi_income < target_persons.opm_pov_thresh
  ubi_total_poor = (target_persons.wgt_numper * target_persons.ubi_poor).sum()
  ubi_percent_poor = ubi_total_poor / target_pop
  percent_ubi_poor = (ubi_percent_poor * 100).round(1)

  target_persons['ed_debt_income'] = target_persons.edn_inst + target_persons.income
  target_persons['ed_debt_poor'] = target_persons.ed_debt_income < target_persons.opm_pov_thresh
  ed_debt_total_poor = (target_persons.wgt_numper * target_persons.ed_debt_poor).sum()
  ed_debt_percent_poor = ed_debt_total_poor / target_pop
  percent_ed_debt_poor = (ed_debt_percent_poor * 100).round(1)

  # Calculate poverty gap
  target_persons['original_poverty_gap'] = target_persons.opm_pov_thresh - target_persons.income
  original_gap_billions = (((target_persons.original_poor * target_persons.original_poverty_gap * target_persons.wgt_numper).sum()) / 1e9).round(1)

  target_persons['ubi_poverty_gap'] = target_persons.opm_pov_thresh - target_persons.ubi_income
  ubi_gap_billions = (((target_persons.ubi_poor * target_persons.ubi_poverty_gap * target_persons.wgt_numper).sum()) / 1e9).round(1)

  target_persons['ed_debt_poverty_gap'] = target_persons.opm_pov_thresh - target_persons.ed_debt_income
  ed_debt_gap_billions = (((target_persons.ed_debt_poor * target_persons.ed_debt_poverty_gap * target_persons.wgt_numper).sum()) / 1e9).round(1)

  # Convert and round
  ed_debt_change = ed_debt_change.astype(int)
  ubi_change = ubi_change.astype(int)

  return pd.Series([ed_debt_change, ubi_change, percent_better_off_with_ed, 
                    percent_initial_debt, percent_ubi_debt, 
                    percent_no_ed_debt_debt, percent_original_poor, 
                    percent_ubi_poor, percent_ed_debt_poor,
                    original_gap_billions, ubi_gap_billions, ed_debt_gap_billions])

In [103]:
def networth_row(row):
  return networth_fun(row.networth_pp_quintile2)

In [104]:
networth[['ed_debt_change', 'ubi_change', 'percent_better_off_with_ed',
        'in_debt', 'in_debt_ubi', 'in_debt_no_ed_debt',
        'percent_original_poor', 'percent_ubi_poor', 'percent_ed_debt_poor', 
     'original_gap_billions', 'ubi_gap_billions', 'ed_debt_gap_billions']] = networth.apply(networth_row, axis=1)

In [105]:
def poor_fun(original_poor):
  target_persons = scf[scf.original_poor==original_poor].copy(deep=True)

  # Calculate average benefit by reform
  start = mdf.weighted_mean(target_persons, 'networth_pp', 'wgt_numper')
  ed_debt = mdf.weighted_mean(target_persons, 'no_debt_networth_pp', 'wgt_numper')
  ubi = mdf.weighted_mean(target_persons, 'ubi_networth_pp', 'wgt_numper')

  ed_debt_change = ed_debt - start
  ubi_change = ubi - start

  # Calculate percent better off with ed debt cancellation
  target_persons['better_with_ed'] = target_persons.ubi_networth_pp < target_persons.no_debt_networth_pp
  total_better_off_with_ed = (target_persons.better_with_ed * target_persons.wgt_numper).sum()
  target_pop = target_persons.wgt_numper.sum()
  percent_better_off_with_ed = (total_better_off_with_ed / target_pop * 100).round(1)

  # Calculate percent out of debt
  target_persons['initial_debt'] = target_persons.networth_pp < 0
  target_persons['ubi_debt'] = target_persons.ubi_networth_pp < 0
  target_persons['no_ed_debt_debt'] = target_persons.no_debt_networth_pp < 0

  percent_initial_debt = (((target_persons.initial_debt 
                           * target_persons.wgt_numper).sum()) / target_pop 
                          * 100).round(1)
  percent_ubi_debt = (((target_persons.ubi_debt 
                           * target_persons.wgt_numper).sum()) / target_pop
                      * 100).round(1)
  percent_no_ed_debt_debt = (((target_persons.no_ed_debt_debt 
                           * target_persons.wgt_numper).sum()) / target_pop
                           * 100).round(1)

  # Calculate percent poor
  total_original_poor = (target_persons.original_poor * target_persons.wgt_numper).sum()
  percent_original_poor = (total_original_poor / target_pop * 100).round(1)

  target_persons['ubi_income'] = target_persons.total_ubi + target_persons.income
  target_persons['ubi_poor'] = target_persons.ubi_income < target_persons.opm_pov_thresh
  ubi_total_poor = (target_persons.wgt_numper * target_persons.ubi_poor).sum()
  ubi_percent_poor = ubi_total_poor / target_pop
  percent_ubi_poor = (ubi_percent_poor * 100).round(1)

  target_persons['ed_debt_income'] = target_persons.edn_inst + target_persons.income
  target_persons['ed_debt_poor'] = target_persons.ed_debt_income < target_persons.opm_pov_thresh
  ed_debt_total_poor = (target_persons.wgt_numper * target_persons.ed_debt_poor).sum()
  ed_debt_percent_poor = ed_debt_total_poor / target_pop
  percent_ed_debt_poor = (ed_debt_percent_poor * 100).round(1)

  # Calculate poverty gap
  target_persons['original_poverty_gap'] = target_persons.opm_pov_thresh - target_persons.income
  original_gap_billions = (((target_persons.original_poor * target_persons.original_poverty_gap * target_persons.wgt_numper).sum()) / 1e9).round(1)

  target_persons['ubi_poverty_gap'] = target_persons.opm_pov_thresh - target_persons.ubi_income
  ubi_gap_billions = (((target_persons.ubi_poor * target_persons.ubi_poverty_gap * target_persons.wgt_numper).sum()) / 1e9).round(1)

  target_persons['ed_debt_poverty_gap'] = target_persons.opm_pov_thresh - target_persons.ed_debt_income
  ed_debt_gap_billions = (((target_persons.ed_debt_poor * target_persons.ed_debt_poverty_gap * target_persons.wgt_numper).sum()) / 1e9).round(1)

  # Convert and round
  ed_debt_change = ed_debt_change.astype(int)
  ubi_change = ubi_change.astype(int)

  return pd.Series([ed_debt_change, ubi_change, percent_better_off_with_ed, 
                    percent_initial_debt, percent_ubi_debt, 
                    percent_no_ed_debt_debt, percent_original_poor, 
                    percent_ubi_poor, percent_ed_debt_poor,
                    original_gap_billions, ubi_gap_billions, ed_debt_gap_billions])

In [106]:
def poor_row(row):
  return poor_fun(row.original_poor)

In [107]:
poor[['ed_debt_change', 'ubi_change', 'percent_better_off_with_ed',
        'in_debt', 'in_debt_ubi', 'in_debt_no_ed_debt',
        'percent_original_poor', 'percent_ubi_poor', 'percent_ed_debt_poor', 
     'original_gap_billions', 'ubi_gap_billions', 'ed_debt_gap_billions']] = poor.apply(poor_row, axis=1)

## Edit Tables

In [108]:
education.sort_values(by=['percent_original_poor'], inplace=True,ascending=False)

In [109]:
age.sort_values(by=['percent_of_total_debt'], inplace=True, ascending=False)

In [110]:
poor.sort_values(by=['percent_of_total_debt'], inplace=True, ascending=True)

In [111]:
poor['original_poor'] = poor['original_poor'].astype(str)
poor['original_poor'].replace({'True': 'In poverty', 'False':'Not in poverty'},inplace=True)

In [112]:
education["edcl"].replace({"Less than high school diploma": ["Less than high<br>school diploma"],
                           "High school diploma": "High school<br>diploma",
                          "Some college": "Some<br>college",
                          "College graduate": "College<br>graduate"}, inplace=True)

In [113]:
all['index'] = all['index'].replace([0],'All')

In [114]:
all = all.rename(columns={"index": "race"})
race = pd.concat([race, all])

In [115]:
all = all.rename(columns={"race": "edcl"})
education = pd.concat([education, all])

In [116]:
all = all.rename(columns={"edcl": "income_pp_quintile"})
income = pd.concat([income, all])

In [117]:
all = all.rename(columns={"income_pp_quintile": "agecl"})
age = pd.concat([age, all])

In [118]:
all = all.rename(columns={"agecl": "networth_pp_quintile2"})
networth = pd.concat([networth, all])

In [119]:
all = all.rename(columns={"networth_pp_quintile2": "original_poor"})
poor = pd.concat([poor, all])

In [120]:
race = race.astype({"ubi_change":'int', "ed_debt_change":'int'})
education = education.astype({"ubi_change":'int', "ed_debt_change":'int'})
age = age.astype({"ubi_change":'int', "ed_debt_change":'int'})
income = income.astype({"ubi_change":'int', "ed_debt_change":'int'})
networth = networth.astype({"ubi_change":'int', "ed_debt_change":'int'})
poor = poor.astype({"ubi_change":'int', "ed_debt_change":'int'})

In [121]:
race['mean_debt'] = race.apply(lambda x: "{:,}".format(x['mean_debt']), axis=1)
race['ed_debt_change'] = race.apply(lambda x: "{:,}".format(x['ed_debt_change']), axis=1)
race['ubi_change'] = race.apply(lambda x: "{:,}".format(x['ubi_change']), axis=1)

education['mean_debt'] = education.apply(lambda x: "{:,}".format(x['mean_debt']), axis=1)
education['ed_debt_change'] = education.apply(lambda x: "{:,}".format(x['ed_debt_change']), axis=1)
education['ubi_change'] = education.apply(lambda x: "{:,}".format(x['ubi_change']), axis=1)

age['mean_debt'] = age.apply(lambda x: "{:,}".format(x['mean_debt']), axis=1)
age['ed_debt_change'] = age.apply(lambda x: "{:,}".format(x['ed_debt_change']), axis=1)
age['ubi_change'] = age.apply(lambda x: "{:,}".format(x['ubi_change']), axis=1)

income['mean_debt'] = income.apply(lambda x: "{:,}".format(x['mean_debt']), axis=1)
income['ed_debt_change'] = income.apply(lambda x: "{:,}".format(x['ed_debt_change']), axis=1)
income['ubi_change'] = income.apply(lambda x: "{:,}".format(x['ubi_change']), axis=1)

networth['mean_debt'] = networth.apply(lambda x: "{:,}".format(x['mean_debt']), axis=1)
networth['ed_debt_change'] = networth.apply(lambda x: "{:,}".format(x['ed_debt_change']), axis=1)
networth['ubi_change'] = networth.apply(lambda x: "{:,}".format(x['ubi_change']), axis=1)

poor['mean_debt'] = poor.apply(lambda x: "{:,}".format(x['mean_debt']), axis=1)
poor['ed_debt_change'] = poor.apply(lambda x: "{:,}".format(x['ed_debt_change']), axis=1)
poor['ubi_change'] = poor.apply(lambda x: "{:,}".format(x['ubi_change']), axis=1)

In [122]:
race.ed_debt_change_50 = race.ed_debt_change_50.astype(int)

education.ed_debt_change_50 = education.ed_debt_change_50.astype(int)

age.ed_debt_change_50 = age.ed_debt_change_50.astype(int)

income.ed_debt_change_50 = income.ed_debt_change_50.astype(int)

networth.ed_debt_change_50 = networth.ed_debt_change_50.astype(int)

poor.ed_debt_change_50 = poor.ed_debt_change_50.astype(int)

race.ubi_change_50 = race.ubi_change_50.astype(int)

education.ubi_change_50 = education.ubi_change_50.astype(int)

age.ubi_change_50 = age.ubi_change_50.astype(int)

income.ubi_change_50 = income.ubi_change_50.astype(int)

networth.ubi_change_50 = networth.ubi_change_50.astype(int)

poor.ubi_change_50 = poor.ubi_change_50.astype(int)

race['ubi_change_50'] = race.apply(lambda x: "{:,}".format(x['ubi_change_50']), axis=1)

education['ubi_change_50'] = education.apply(lambda x: "{:,}".format(x['ubi_change_50']), axis=1)

age['ubi_change_50'] = age.apply(lambda x: "{:,}".format(x['ubi_change_50']), axis=1)

income['ubi_change_50'] = income.apply(lambda x: "{:,}".format(x['ubi_change_50']), axis=1)

networth['ubi_change_50'] = networth.apply(lambda x: "{:,}".format(x['ubi_change_50']), axis=1)

poor['ubi_change_50'] = poor.apply(lambda x: "{:,}".format(x['ubi_change_50']), axis=1)

race['ed_debt_change_50'] = race.apply(lambda x: "{:,}".format(x['ed_debt_change_50']), axis=1)

education['ed_debt_change_50'] = education.apply(lambda x: "{:,}".format(x['ed_debt_change_50']), axis=1)

age['ed_debt_change_50'] = age.apply(lambda x: "{:,}".format(x['ed_debt_change_50']), axis=1)

income['ed_debt_change_50'] = income.apply(lambda x: "{:,}".format(x['ed_debt_change_50']), axis=1)

networth['ed_debt_change_50'] = networth.apply(lambda x: "{:,}".format(x['ed_debt_change_50']), axis=1)

poor['ed_debt_change_50'] = poor.apply(lambda x: "{:,}".format(x['ed_debt_change_50']), axis=1)

In [123]:
race.ed_debt_change_10 = race.ed_debt_change_10.astype(int)

education.ed_debt_change_10 = education.ed_debt_change_10.astype(int)

age.ed_debt_change_10 = age.ed_debt_change_10.astype(int)

income.ed_debt_change_10 = income.ed_debt_change_10.astype(int)

networth.ed_debt_change_10 = networth.ed_debt_change_10.astype(int)

poor.ed_debt_change_10 = poor.ed_debt_change_10.astype(int)

race.ubi_change_10 = race.ubi_change_10.astype(int)

education.ubi_change_10 = education.ubi_change_10.astype(int)

age.ubi_change_10 = age.ubi_change_10.astype(int)

income.ubi_change_10 = income.ubi_change_10.astype(int)

networth.ubi_change_10 = networth.ubi_change_10.astype(int)

poor.ubi_change_10 = poor.ubi_change_10.astype(int)

race['ubi_change_10'] = race.apply(lambda x: "{:,}".format(x['ubi_change_10']), axis=1)

education['ubi_change_10'] = education.apply(lambda x: "{:,}".format(x['ubi_change_10']), axis=1)

age['ubi_change_10'] = age.apply(lambda x: "{:,}".format(x['ubi_change_10']), axis=1)

income['ubi_change_10'] = income.apply(lambda x: "{:,}".format(x['ubi_change_10']), axis=1)

networth['ubi_change_10'] = networth.apply(lambda x: "{:,}".format(x['ubi_change_10']), axis=1)

poor['ubi_change_10'] = poor.apply(lambda x: "{:,}".format(x['ubi_change_10']), axis=1)

race['ed_debt_change_10'] = race.apply(lambda x: "{:,}".format(x['ed_debt_change_10']), axis=1)

education['ed_debt_change_10'] = education.apply(lambda x: "{:,}".format(x['ed_debt_change_10']), axis=1)

age['ed_debt_change_10'] = age.apply(lambda x: "{:,}".format(x['ed_debt_change_10']), axis=1)

income['ed_debt_change_10'] = income.apply(lambda x: "{:,}".format(x['ed_debt_change_10']), axis=1)

networth['ed_debt_change_10'] = networth.apply(lambda x: "{:,}".format(x['ed_debt_change_10']), axis=1)

poor['ed_debt_change_10'] = poor.apply(lambda x: "{:,}".format(x['ed_debt_change_10']), axis=1)

## Tables

In [124]:
all.to_csv(r'all_debt_ubi')

In [125]:
race.to_csv(r'race_debt_ubi')

In [126]:
education.to_csv(r'education_debt_ubi')

In [127]:
income.to_csv(r'income_debt_ubi')

In [128]:
age.to_csv(r'age_debt_ubi')

In [129]:
networth.to_csv(r'networth_debt_ubi')

In [130]:
poor.to_csv(r'poor_debt_ubi')

## Change in Gini

In [131]:
start_gini = mdf.gini(scf, 'networth_pp', w='wgt_numper')
start_gini

0.8580552533769346

In [132]:
ubi_gini = mdf.gini(scf, 'ubi_networth_pp', w='wgt_numper')
ubi_gini

0.8482138088314078

In [133]:
ed_gini = mdf.gini(scf, 'no_debt_networth_pp', w='wgt_numper')
ed_gini

0.8444108755674202

In [134]:
start_gini_inc = mdf.gini(scf, 'income_pp', w='wgt_numper')
start_gini_inc

0.5674710044436

In [135]:
ubi_gini_inc = mdf.gini(scf, 'ubi_income_pp', w='wgt_numper')
ubi_gini_inc

0.5246819551763151

In [136]:
no_debt_gini_inc = mdf.gini(scf, 'no_debt_income_pp', w='wgt_numper')
no_debt_gini_inc

0.5588274060412531

In [137]:
networth

,networth_pp_quintile2,mean_debt,has_debt,population,total_debt_held,percent_has_debt,percent_of_total_debt,percent_pop,ratio,ed_debt_change_50,ubi_change_50,percent_better_off_with_ed_50,in_debt_50,in_debt_ubi_50,in_debt_no_ed_debt_50,percent_ubi_poor_50,percent_ed_debt_poor_50,ubi_gap_billions_50,ed_debt_gap_billions_50,ed_debt_change_10,ubi_change_10,percent_better_off_with_ed_10,in_debt_10,in_debt_ubi_10,in_debt_no_ed_debt_10,percent_ubi_poor_10,percent_ed_debt_poor_10,ubi_gap_billions_10,ed_debt_gap_billions_10,ed_debt_change,ubi_change,percent_better_off_with_ed,in_debt,in_debt_ubi,in_debt_no_ed_debt,percent_original_poor,percent_ubi_poor,percent_ed_debt_poor,original_gap_billions,ubi_gap_billions,ed_debt_gap_billions
0,0-20,"10,124",2.533623e+07,6.099631e+07,6.175778e+11,41.5,0.554035,0.199996,2.77,"5,586","2,337",35.0,49.0,30.2,20.1,13.3,23.2,38.1,114.7,"1,618",771,38.9,49.0,30.2,20.1,23.0,24.0,92.8,117.4,"10,124","3,654",32.1,49.0,30.2,20.1,27.9,6.9,23.2,136.2,15.0,114.7
1,20-40,"2,042",1.649594e+07,6.099471e+07,1.245606e+11,27.0,0.111745,0.199990,0.56,"1,680","2,337",17.1,0.0,0.0,0.0,5.0,10.2,13.9,41.2,653,771,23.4,0.0,0.0,0.0,9.8,11.0,34.8,43.3,"2,042","3,654",14.9,0.0,0.0,0.0,12.1,2.7,10.2,52.6,5.8,41.2
2,40-60,"2,418",1.606503e+07,6.098782e+07,1.474932e+11,26.3,0.132318,0.199968,0.66,"1,889","2,337",18.0,0.0,0.0,0.0,2.9,4.3,7.6,18.5,715,771,25.1,0.0,0.0,0.0,4.4,4.3,15.2,18.5,"2,418","3,654",14.8,0.0,0.0,0.0,5.0,1.7,4.3,20.1,3.8,18.5
3,60-80,"2,229",1.114570e+07,6.099198e+07,1.359943e+11,18.3,0.122002,0.199981,0.61,"1,583","2,337",12.9,0.0,0.0,0.0,1.8,3.8,7.2,16.1,549,771,16.4,0.0,0.0,0.0,3.4,3.8,12.7,16.1,"2,229","3,654",11.2,0.0,0.0,0.0,3.9,1.4,3.8,17.2,4.6,16.1
4,80-100,"1,459",5.613266e+06,6.101745e+07,8.906459e+10,9.2,0.079901,0.200065,0.40,948,"2,337",7.4,0.0,0.0,0.0,1.1,1.3,6.3,9.5,319,771,8.9,0.0,0.0,0.0,1.3,1.3,8.4,9.5,"1,459","3,654",6.4,0.0,0.0,0.0,1.3,1.0,1.3,9.5,4.6,9.5
0,All,"3,654",7.465616e+07,3.049883e+08,1.114690e+12,24.5,1.000000,1.000000,1.00,"2,337","2,337",18.1,9.8,6.0,4.0,4.8,8.6,73.0,200.0,771,771,22.5,9.8,6.0,4.0,8.4,8.9,163.8,204.8,"3,654","3,654",15.9,9.8,6.0,4.0,10.0,2.7,8.6,235.5,33.8,200.0


In [ ]:
race